In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

import warnings
import time
import joblib
import os
import itertools

from tqdm.notebook import tqdm

from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.tbats import TBATS
from sktime.forecasting.base import ForecastingHorizon

from prophet import Prophet
from orbit.models import DLT
from statsmodels.tsa.api import ExponentialSmoothing

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder


# Настройки
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)
sns.set_style('darkgrid')
warnings.filterwarnings("ignore")

## Загрузка и разбиение данных

In [3]:
# --- Загрузка данных ---
STORE_ID = 'STORE_1'
processed_data_path = f'data/processed_{STORE_ID}_data.csv'
df = pd.read_csv(processed_data_path, parse_dates=['date'])

print(f"Загружены обработанные данные для {STORE_ID}: {df.shape}")
print("\nПервые 5 строк:")
print(df.head())
print("\nИнформация о данных:")
df.info()
print("\nПроверка пропусков (на всякий случай):")
print(df.isnull().sum().sum()) # Должен быть 0

Загружены обработанные данные для STORE_1: (27285, 13)

Первые 5 строк:
       item_id  date_id  cnt       date  wm_yr_wk    weekday  wday  month  year event_name_1 event_type_1  cashback  sell_price
0  STORE_1_064        1    0 2011-01-29     11101   Saturday     1      1  2011      NoEvent       NoType         0        2.54
1  STORE_1_064        2    1 2011-01-30     11101     Sunday     2      1  2011      NoEvent       NoType         0        2.54
2  STORE_1_064        3    0 2011-01-31     11101     Monday     3      1  2011      NoEvent       NoType         0        2.54
3  STORE_1_064        4    0 2011-02-01     11101    Tuesday     4      2  2011      NoEvent       NoType         0        2.54
4  STORE_1_064        5    0 2011-02-02     11101  Wednesday     5      2  2011      NoEvent       NoType         1        2.54

Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27285 entries, 0 to 27284
Data columns (total 13 columns):
 #   Column        Non-Null C

In [4]:
# --- Параметры прогнозирования ---
FH_WEEK = 7
FH_MONTH = 30
FH_QUARTER = 90
TEST_SIZE = FH_QUARTER # Размер тестовой выборки равен максимальному горизонту

# --- Разбиение данных ---

# Определяем точку разделения
split_date = df['date'].max() - pd.Timedelta(days=TEST_SIZE -1) # -1 т.к. включительно

# Создаем обучающую и тестовую выборки
train_df = df[df['date'] < split_date].copy()
test_df = df[df['date'] >= split_date].copy()

print(f"\n--- Разбиение данных ---")
print(f"Дата разделения: {split_date.date()}")
print(f"Размер обучающей выборки (train_df): {train_df.shape}")
print(f"Временной диапазон train: {train_df['date'].min().date()} - {train_df['date'].max().date()}")
print(f"Размер тестовой выборки (test_df): {test_df.shape}")
print(f"Временной диапазон test: {test_df['date'].min().date()} - {test_df['date'].max().date()}")

# Проверим, что в тесте ровно TEST_SIZE дней для каждого товара
test_days_per_item = test_df.groupby('item_id')['date'].nunique()
print(f"\nУникальных дней в тесте на товар (min/max): {test_days_per_item.min()}/{test_days_per_item.max()}")
if test_days_per_item.min() != TEST_SIZE or test_days_per_item.max() != TEST_SIZE:
    print("ПРЕДУПРЕЖДЕНИЕ: Количество дней в тестовой выборке не совпадает с TEST_SIZE для некоторых товаров!")


--- Разбиение данных ---
Дата разделения: 2015-10-24
Размер обучающей выборки (train_df): (25935, 13)
Временной диапазон train: 2011-01-29 - 2015-10-23
Размер тестовой выборки (test_df): (1350, 13)
Временной диапазон test: 2015-10-24 - 2016-01-21

Уникальных дней в тесте на товар (min/max): 90/90


**Выбор метрик для задачи:**

*   **MAE и RMSE:** Хорошие основные метрики для оценки абсолютной ошибки в единицах товара. MAE более робастна, RMSE сильнее штрафует большие промахи. Будем использовать обе.
*   **MAPE/sMAPE:** Из-за наличия нулевых продаж в данных, стандартный MAPE использовать рискованно. sMAPE является более безопасной альтернативой для оценки относительной ошибки.
*   **R²:** Включим для общей информации, но не будем делать на нее основной упор.

Создадим функцию для расчета этих метрик.

In [5]:
def calculate_metrics(y_true, y_pred):
    """Рассчитывает MAE, RMSE, sMAPE и R2."""
    # Убедимся, что нет NaN/inf в прогнозах, заменим их на 0 (или другое значение)
    y_pred = np.nan_to_num(y_pred)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # Расчет sMAPE с защитой от деления на ноль в знаменателе
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Избегаем деления на ноль, где и числитель, и знаменатель равны 0
    smape = np.mean(np.divide(numerator, denominator, out=np.zeros_like(numerator, dtype=float), where=denominator!=0)) * 100
    # Если нужно другое поведение при 0/0 (например, считать ошибку 0), можно изменить 'out'

    r2 = r2_score(y_true, y_pred)

    return {
        'MAE': mae,
        'RMSE': rmse,
        'sMAPE': smape,
        'R2': r2
    }

## Обучение стат. моделей

### Наивный прогноз

In [6]:
# --- Baseline: Сезонный наивный прогноз (сдвиг на 7 дней) ---

def seasonal_naive_forecast(train_data, fh):
    """Генерирует сезонный наивный прогноз (сдвиг на 7 дней) для всех товаров."""
    forecasts = []
    unique_items = train_data['item_id'].unique()
    last_train_date = train_data['date'].max()

    for item in unique_items:
        item_train_data = train_data[train_data['item_id'] == item].set_index('date').sort_index()
        # Берем последние 7 дней из трейна как основу для прогноза
        last_week_values = item_train_data['cnt'].iloc[-7:].values

        # Повторяем значения последней недели нужное количество раз
        reps = int(np.ceil(fh / 7))
        item_preds_raw = np.tile(last_week_values, reps)[:fh] # Берем только нужную длину fh

        # Создаем индекс дат для прогноза
        pred_dates = pd.date_range(start=last_train_date + pd.Timedelta(days=1), periods=fh, freq='D')

        # Создаем DataFrame для прогноза этого товара
        item_forecast_df = pd.DataFrame({
            'date': pred_dates,
            'item_id': item,
            'yhat_baseline': item_preds_raw # Назовем прогноз 'yhat_baseline'
        })
        forecasts.append(item_forecast_df)

    # Объединяем прогнозы для всех товаров
    baseline_forecast_df = pd.concat(forecasts).reset_index(drop=True)
    return baseline_forecast_df

# Генерируем baseline прогнозы на тестовый период (90 дней)
baseline_preds = seasonal_naive_forecast(train_df, FH_QUARTER)

print("\n--- Baseline прогноз (Seasonal Naive, FH=90) ---")
# print(baseline_preds.head())
# print(baseline_preds.shape)

# Оценка Baseline модели
# Сначала объединим прогнозы с реальными значениями из test_df
test_merged_baseline = pd.merge(test_df[['date', 'item_id', 'cnt']],
                                baseline_preds,
                                on=['date', 'item_id'],
                                how='left')


baseline_results_summary = {}
min_test_date = test_merged_baseline['date'].min()

# Горизонты для оценки
horizons = {'Week': FH_WEEK, 'Month': FH_MONTH, 'Quarter': FH_QUARTER}

# Рассчитываем метрики для каждого горизонта
for name, days in horizons.items():
    # Фильтруем данные для текущего горизонта
    test_horizon = test_merged_baseline[test_merged_baseline['date'] < min_test_date + pd.Timedelta(days=days)]
    # Считаем метрики по каждому товару и усредняем
    horizon_metrics_df = test_horizon.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_baseline'])))
    # Сохраняем средние значения
    baseline_results_summary[name] = horizon_metrics_df.mean()

# Создаем и выводим итоговую таблицу
baseline_summary_df = pd.DataFrame(baseline_results_summary)

print("\n--- Baseline метрики по горизонтам прогнозирования ---")
print(baseline_summary_df)

# Дополнительно: Вывод метрик по товарам для самого длинного горизонта (квартал)
print("\n--- Метрики Baseline по товарам (Квартал) ---")
baseline_metrics_quarter_df = test_merged_baseline.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_baseline'])))
print(baseline_metrics_quarter_df)


--- Baseline прогноз (Seasonal Naive, FH=90) ---

--- Baseline метрики по горизонтам прогнозирования ---
            Week      Month    Quarter
MAE     5.180952   5.695556   7.934074
RMSE    6.515400   7.371659  10.239756
sMAPE  69.216369  70.952425  78.637057
R2     -0.820453  -0.687139  -0.657725

--- Метрики Baseline по товарам (Квартал) ---
                   MAE       RMSE       sMAPE        R2
item_id                                                
STORE_1_064   0.211111   0.567646   33.333333 -0.160516
STORE_1_065   0.733333   1.358103   94.920635 -0.215029
STORE_1_090  44.422222  57.453750   95.936457 -0.581973
STORE_1_252   7.644444   9.473941   46.762621 -0.520806
STORE_1_325   4.244444   5.189733  105.340298 -0.346625
STORE_1_339   3.200000   4.071582  121.984959 -1.272696
STORE_1_376   0.711111   1.183216   89.888889 -0.254980
STORE_1_546   1.855556   2.433562   76.023162 -0.446579
STORE_1_547  13.266667  17.287761  146.038597 -1.043643
STORE_1_555   7.988889  10.115445   

### Обучение и прогнозы AutoARIMA

In [ ]:
# Обучение AutoARIMA

# Словарь для хранения обученных моделей и времени обучения
autoarima_models = {}
autoarima_fit_times = {}

print(f"--- Обучение AutoARIMA (sp=7) для {len(train_df['item_id'].unique())} товаров ---")
start_total_fit_time = time.time()

for item_id in tqdm(train_df['item_id'].unique(), desc="Обучение AutoARIMA"):
    start_item_fit_time = time.time()
    # Готовим данные для sktime (Series с DatetimeIndex)
    item_train_series = train_df[train_df['item_id'] == item_id].set_index('date')['cnt'].sort_index()

    # Проверяем на наличие достаточного количества данных (хотя бы 2 сезона)
    if len(item_train_series) < 2 * 7:
        print(f"Пропуск {item_id}: недостаточно данных ({len(item_train_series)} точек)")
        autoarima_models[item_id] = None
        autoarima_fit_times[item_id] = None
        continue

    # Инициализируем модель AutoARIMA
    # sp=7 для недельной сезонности
    # suppress_warnings=True, чтобы избежать множества предупреждений от pmdarima
    # error_action='ignore', чтобы пропустить ряды, где модель не сходится
    forecaster = AutoARIMA(sp=7, suppress_warnings=True, error_action='ignore', maxiter=10) # maxiter можно увеличить, если нужно больше попыток

    try:
        forecaster.fit(y=item_train_series)
        autoarima_models[item_id] = forecaster
        end_item_fit_time = time.time()
        autoarima_fit_times[item_id] = end_item_fit_time - start_item_fit_time
    except Exception as e:
        print(f"Ошибка обучения AutoARIMA для {item_id}: {e}")
        autoarima_models[item_id] = None
        autoarima_fit_times[item_id] = None

end_total_fit_time = time.time()
successful_fits = sum(1 for model in autoarima_models.values() if model is not None)
print(f"\nОбучение AutoARIMA завершено.")
print(f"Общее время обучения: {end_total_fit_time - start_total_fit_time:.2f} сек.")
print(f"Успешно обучено моделей: {successful_fits} из {len(train_df['item_id'].unique())}")

# Можно посмотреть среднее время обучения на 1 товар
valid_fit_times = [t for t in autoarima_fit_times.values() if t is not None]
if valid_fit_times:
    print(f"Среднее время обучения на 1 модель: {np.mean(valid_fit_times):.2f} сек.")

--- Обучение AutoARIMA (sp=7) для 15 товаров ---


Обучение AutoARIMA:   0%|          | 0/15 [00:00<?, ?it/s]


Обучение AutoARIMA завершено.
Общее время обучения: 525.08 сек.
Успешно обучено моделей: 15 из 15
Среднее время обучения на 1 модель: 35.00 сек.


In [76]:
# Прогнозирование и Оценка AutoARIMA

# Список для хранения прогнозов
autoarima_forecasts = []
autoarima_predict_times = {}

print("\n--- Генерация прогнозов AutoARIMA (на 90 дней) ---")
start_total_predict_time = time.time()

# Определяем горизонт прогнозирования для sktime
# Это будут шаги относительно конца обучающей выборки
fh = ForecastingHorizon(np.arange(1, FH_QUARTER + 1), is_relative=True)

# Даты тестового периода (нужны для создания итогового DataFrame)
test_start_date = test_df['date'].min()
test_end_date = test_df['date'].max()
pred_dates_index = pd.date_range(start=test_start_date, end=test_end_date, freq='D')


for item_id, model in tqdm(autoarima_models.items(), desc="Прогнозирование AutoARIMA"):
    # Заготовка DataFrame для товара
    item_forecast_df = pd.DataFrame({'date': pred_dates_index, 'item_id': item_id})

    if model is not None:
        start_item_predict_time = time.time()
        try:
            # Генерируем прогноз
            y_pred = model.predict(fh=fh)
            # Присваиваем правильные даты
            y_pred.index = pred_dates_index
            item_forecast_df['yhat_autoarima'] = y_pred.values

            end_item_predict_time = time.time()
            autoarima_predict_times[item_id] = end_item_predict_time - start_item_predict_time

        except Exception as e:
            print(f"Ошибка прогнозирования AutoARIMA для {item_id}: {e}")
            item_forecast_df['yhat_autoarima'] = np.nan # Заполняем NaN при ошибке
            autoarima_predict_times[item_id] = None
    else:
        item_forecast_df['yhat_autoarima'] = np.nan # Заполняем NaN, если модель не обучилась
        autoarima_predict_times[item_id] = None

    autoarima_forecasts.append(item_forecast_df)

# Объединяем прогнозы
autoarima_forecast_df = pd.concat(autoarima_forecasts).reset_index(drop=True)
# Обрабатываем NaN и отрицательные значения
autoarima_forecast_df['yhat_autoarima'] = autoarima_forecast_df['yhat_autoarima'].clip(lower=0).fillna(0)


end_total_predict_time = time.time()
print(f"\nОбщее время прогнозирования AutoARIMA: {end_total_predict_time - start_total_predict_time:.2f} сек.")
print(f"Размер датафрейма прогнозов: {autoarima_forecast_df.shape}")


--- Генерация прогнозов AutoARIMA (на 90 дней) ---


Прогнозирование AutoARIMA:   0%|          | 0/15 [00:00<?, ?it/s]

/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/mnt/c/Users/edavletshin/courses/!ti


Общее время прогнозирования AutoARIMA: 0.86 сек.
Размер датафрейма прогнозов: (1350, 3)


/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [75]:
# --- Оценка AutoARIMA модели ---
test_merged_autoarima = pd.merge(test_df[['date', 'item_id', 'cnt']],
                                   autoarima_forecast_df,
                                   on=['date', 'item_id'],
                                   how='left')

# --- Сводная таблица метрик AutoARIMA ---
autoarima_results_summary = {}
min_test_date = test_merged_autoarima['date'].min()

# Горизонты для оценки
horizons = {'Week': FH_WEEK, 'Month': FH_MONTH, 'Quarter': FH_QUARTER}

# Рассчитываем метрики для каждого горизонта
for name, days in horizons.items():
    test_horizon = test_merged_autoarima[test_merged_autoarima['date'] <= min_test_date + pd.Timedelta(days=days-1)]
    horizon_metrics_df = test_horizon.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_autoarima'])))
    autoarima_results_summary[name] = horizon_metrics_df.mean()

# Создаем и выводим итоговую таблицу
autoarima_summary_df = pd.DataFrame(autoarima_results_summary)

print("\n--- Сводная таблица метрик AutoARIMA ---")
print(autoarima_summary_df)

# Дополнительно: Вывод метрик по товарам для самого длинного горизонта (квартал)
print("\n--- Метрики AutoARIMA по товарам (Квартал) ---")
autoarima_metrics_quarter_df = test_merged_autoarima.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_autoarima'])))
print(autoarima_metrics_quarter_df)


--- Сводная таблица метрик AutoARIMA ---
            Week      Month    Quarter
MAE     4.621129   4.911177   7.141438
RMSE    5.297589   5.838641   9.063509
sMAPE  79.628187  81.025704  85.591646
R2     -0.021185   0.035009  -0.072786

--- Метрики AutoARIMA по товарам (Квартал) ---
                   MAE       RMSE       sMAPE        R2
item_id                                                
STORE_1_064   0.308411   0.530896  192.389462 -0.015112
STORE_1_065   0.741646   1.263027  169.073342 -0.050863
STORE_1_090  49.851881  62.232142   97.052127 -0.856060
STORE_1_252   6.463357   7.920427   42.856258 -0.062942
STORE_1_325   3.562628   4.501222   75.023497 -0.013019
STORE_1_339   2.234007   2.930407   79.991313 -0.177256
STORE_1_376   0.768906   1.060925  175.856138 -0.008970
STORE_1_546   1.698646   2.030667   62.306527 -0.007244
STORE_1_547  10.529573  13.789116   99.457974 -0.300171
STORE_1_555   6.451262   7.925201   28.910318  0.059428
STORE_1_584   1.423710   2.252330  107.8237

### Обучение и прогнозы ETS

In [ ]:
# --- Обучение ETS ---

ets_models = {}
ets_fit_times = {}

print("--- Обучение ETS ---")
start_total_time = time.time()

# Оборачиваем цикл в tqdm
for item_id in tqdm(train_df['item_id'].unique(), desc="Обучение ETS"):
    item_train_series = train_df[train_df['item_id'] == item_id].set_index('date')['cnt']

    start_item_time = time.time()
    try:
        ets_model = ExponentialSmoothing(item_train_series,
                                         trend='add',
                                         seasonal='add',
                                         seasonal_periods=7,
                                         damped_trend=True,
                                         initialization_method='estimated'
                                         ).fit()
        ets_models[item_id] = ets_model
        end_item_time = time.time()
        ets_fit_times[item_id] = end_item_time - start_item_time

    except Exception as e:
        # В случае ошибки можно добавить логирование или тихий пропуск
        # print(f"  Ошибка обучения ETS для {item_id}: {e}") # Убрали вывод ошибки для чистоты tqdm
        ets_models[item_id] = None
        ets_fit_times[item_id] = None

end_total_time = time.time()
print(f"\nОбщее время обучения ETS: {end_total_time - start_total_time:.2f} сек.")

successful_fits = sum(1 for model in ets_models.values() if model is not None)
print(f"Успешно обучено моделей ETS: {successful_fits} из {len(train_df['item_id'].unique())}")

--- Обучение ETS моделей ---


Обучение ETS:   0%|          | 0/15 [00:00<?, ?it/s]

/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/mnt/c/U


Общее время обучения ETS: 3.56 сек.
Успешно обучено моделей ETS: 15 из 15


In [8]:
# --- Генерация прогнозов ETS ---

ets_forecasts = []
ets_predict_times = {}

print("--- Генерация прогнозов ETS (на 90 дней) ---")
start_total_predict_time = time.time()

# Определяем начало и конец периода прогнозирования
start_pred_date = test_df['date'].min()
end_pred_date = test_df['date'].max()
# Создаем полный индекс дат для прогноза (на случай ошибок в predict)
pred_dates_index = pd.date_range(start=start_pred_date, end=end_pred_date, freq='D')

for item_id, model in tqdm(ets_models.items(), desc="Прогнозирование ETS"):
    item_forecast_df = pd.DataFrame({'date': pred_dates_index, 'item_id': item_id})
    if model is not None:
        start_item_predict_time = time.time()
        try:
            # Используем forecast вместо predict для ETSModel
            item_preds = model.forecast(steps=FH_QUARTER) # Прогноз на FH_QUARTER шагов вперед
            # Проверяем длину прогноза и индекс
            if len(item_preds) == FH_QUARTER:
                 item_preds.index = pred_dates_index # Присваиваем правильный индекс дат
                 item_forecast_df['yhat_ets'] = item_preds.values
            else:
                 # Если длина не совпала, заполняем NaN
                 print(f"Warning: Прогноз для {item_id} имеет неверную длину ({len(item_preds)}), ожидалось {FH_QUARTER}. Заполняем NaN.")
                 item_forecast_df['yhat_ets'] = np.nan

            end_item_predict_time = time.time()
            ets_predict_times[item_id] = end_item_predict_time - start_item_predict_time

        except Exception as e:
            print(f"Ошибка прогнозирования ETS для {item_id}: {e}. Заполняем NaN.")
            item_forecast_df['yhat_ets'] = np.nan
            ets_predict_times[item_id] = None
    else:
         # Если модель не обучилась
         item_forecast_df['yhat_ets'] = np.nan
         ets_predict_times[item_id] = None

    ets_forecasts.append(item_forecast_df)


# Объединяем прогнозы
ets_forecast_df = pd.concat(ets_forecasts).reset_index(drop=True)
# Применяем clip(lower=0) и обрабатываем возможные NaN перед оценкой
ets_forecast_df['yhat_ets'] = ets_forecast_df['yhat_ets'].clip(lower=0).fillna(0) # Заменяем NaN на 0 перед оценкой

end_total_predict_time = time.time()
print(f"\nОбщее время прогнозирования ETS: {end_total_predict_time - start_total_predict_time:.2f} сек.")
print(f"Размер датафрейма прогнозов: {ets_forecast_df.shape}")


# --- Оценка ETS модели ---
test_merged_ets = pd.merge(test_df[['date', 'item_id', 'cnt']],
                           ets_forecast_df,
                           on=['date', 'item_id'],
                           how='left')

# --- Сводная таблица метрик ETS ---
ets_results_summary = {}
min_test_date = test_merged_ets['date'].min()

# Горизонты для оценки
horizons = {'Week': FH_WEEK, 'Month': FH_MONTH, 'Quarter': FH_QUARTER}

# Рассчитываем метрики для каждого горизонта
for name, days in horizons.items():
    # Фильтруем данные для текущего горизонта
    test_horizon = test_merged_ets[test_merged_ets['date'] <= min_test_date + pd.Timedelta(days=days-1)]
    # Считаем метрики по каждому товару и усредняем
    horizon_metrics_df = test_horizon.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_ets'])))
    # Сохраняем средние значения
    ets_results_summary[name] = horizon_metrics_df.mean()

# Создаем и выводим итоговую таблицу
ets_summary_df = pd.DataFrame(ets_results_summary)

print("\n--- Сводная таблица метрик ETS ---")
print(ets_summary_df)

# Дополнительно: Вывод метрик по товарам для самого длинного горизонта (квартал)
print("\n--- Метрики ETS по товарам (Квартал) ---")
ets_metrics_quarter_df = test_merged_ets.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_ets'])))
print(ets_metrics_quarter_df)

--- Генерация прогнозов ETS (на 90 дней) ---


Прогнозирование ETS:   0%|          | 0/15 [00:00<?, ?it/s]


Общее время прогнозирования ETS: 0.20 сек.
Размер датафрейма прогнозов: (1350, 3)

--- Сводная таблица метрик ETS ---
            Week      Month    Quarter
MAE     4.148505   4.785016   7.283846
RMSE    4.871507   5.832586   9.309611
sMAPE  76.182163  79.566157  86.066540
R2      0.097530   0.076141  -0.120005

--- Метрики ETS по товарам (Квартал) ---
                   MAE       RMSE       sMAPE        R2
item_id                                                
STORE_1_064   0.299807   0.527906  189.961938 -0.003711
STORE_1_065   0.744034   1.267290  169.127595 -0.057970
STORE_1_090  49.831912  62.808338   96.690206 -0.890589
STORE_1_252   6.059311   7.578216   40.677824  0.026924
STORE_1_325   3.439252   4.374867   73.419652  0.043056
STORE_1_339   2.151949   2.845997   76.190753 -0.110412
STORE_1_376   0.754685   1.047123  174.257140  0.017112
STORE_1_546   1.558777   1.898724   61.523415  0.119395
STORE_1_547  11.254831  14.858015  111.507831 -0.509556
STORE_1_555   6.604460   8.0

### Обучение и прогнозы Prophet

In [ ]:
# --- Подготовка данных для Prophet ---

# Prophet требует столбцы 'ds' и 'y'
train_prophet_df = train_df[['date', 'item_id', 'cnt']].rename(columns={'date': 'ds', 'cnt': 'y'})
test_prophet_df = test_df[['date', 'item_id', 'cnt']].rename(columns={'date': 'ds', 'cnt': 'y'})

# Подготовка датафрейма с праздниками
# Возьмем информацию из исходного calendar_df
# Нужны столбцы 'holiday' (название) и 'ds' (дата)
original_calendar_df = pd.read_csv('data/shop_sales_dates.csv', parse_dates=['date'])
holidays_df = original_calendar_df[['event_name_1', 'date']].copy()
holidays_df.rename(columns={'event_name_1': 'holiday', 'date': 'ds'}, inplace=True)
# Удаляем дни без событий и дубликаты (если один праздник на несколько дней)
holidays_df = holidays_df.dropna(subset=['holiday'])
holidays_df = holidays_df.drop_duplicates()

# Добавим окна вокруг праздников (например, +/- 1 день)
# Установим окно по умолчанию для всех праздников
holidays_df['lower_window'] = -1
holidays_df['upper_window'] = 1

# Можно задать специфичные окна для важных праздников
holidays_df.loc[holidays_df['holiday'] == 'Christmas', 'lower_window'] = -2
holidays_df.loc[holidays_df['holiday'] == 'Christmas', 'upper_window'] = 0

print("--- DataFrame с праздниками для Prophet ---")
print(holidays_df.head())
print(f"Всего уникальных праздников/событий: {holidays_df['holiday'].nunique()}")


# --- Обучение Prophet моделей ---
prophet_models = {}
prophet_fit_times = {}

print("\n--- Обучение Prophet моделей (по одному на товар) ---")
start_total_time = time.time()

# Оборачиваем цикл в tqdm
for item_id in tqdm(train_prophet_df['item_id'].unique(), desc="Обучение Prophet"):
    item_train_data = train_prophet_df[train_prophet_df['item_id'] == item_id][['ds', 'y']]
    start_item_time = time.time()
    try:
        prophet_model = Prophet(holidays=holidays_df,
                                weekly_seasonality=True,
                                yearly_seasonality=True,
                                daily_seasonality=False,
                                seasonality_mode='additive',
                                uncertainty_samples=0)

        prophet_model.fit(item_train_data)
        prophet_models[item_id] = prophet_model
        end_item_time = time.time()
        prophet_fit_times[item_id] = end_item_time - start_item_time
    except Exception as e:

        prophet_models[item_id] = None
        prophet_fit_times[item_id] = None

end_total_time = time.time()
print(f"\nОбщее время обучения Prophet: {end_total_time - start_total_time:.2f} сек.")

successful_fits_prophet = sum(1 for model in prophet_models.values() if model is not None)
print(f"Успешно обучено моделей Prophet: {successful_fits_prophet} из {len(train_prophet_df['item_id'].unique())}")

Importing plotly failed. Interactive plots will not work.


--- DataFrame с праздниками для Prophet ---
          holiday         ds  lower_window  upper_window
8       SuperBowl 2011-02-06            -1             1
16  ValentinesDay 2011-02-14            -1             1
23  PresidentsDay 2011-02-21            -1             1
39      LentStart 2011-03-09            -1             1
46      LentWeek2 2011-03-16            -1             1
Всего уникальных праздников/событий: 30

--- Обучение Prophet моделей (по одному на товар) ---


Обучение Prophet:   0%|          | 0/15 [00:00<?, ?it/s]

22:02:52 - cmdstanpy - INFO - Chain [1] start processing
22:02:53 - cmdstanpy - INFO - Chain [1] done processing
22:02:53 - cmdstanpy - INFO - Chain [1] start processing
22:02:53 - cmdstanpy - INFO - Chain [1] done processing
22:02:54 - cmdstanpy - INFO - Chain [1] start processing
22:02:54 - cmdstanpy - INFO - Chain [1] done processing
22:02:54 - cmdstanpy - INFO - Chain [1] start processing
22:02:55 - cmdstanpy - INFO - Chain [1] done processing
22:02:55 - cmdstanpy - INFO - Chain [1] start processing
22:02:55 - cmdstanpy - INFO - Chain [1] done processing
22:02:55 - cmdstanpy - INFO - Chain [1] start processing
22:02:56 - cmdstanpy - INFO - Chain [1] done processing
22:02:56 - cmdstanpy - INFO - Chain [1] start processing
22:02:56 - cmdstanpy - INFO - Chain [1] done processing
22:02:57 - cmdstanpy - INFO - Chain [1] start processing
22:02:57 - cmdstanpy - INFO - Chain [1] done processing
22:02:57 - cmdstanpy - INFO - Chain [1] start processing
22:02:57 - cmdstanpy - INFO - Chain [1]


Общее время обучения Prophet: 10.59 сек.
Успешно обучено моделей Prophet: 15 из 15


In [10]:
# --- Генерация прогнозов Prophet ---

prophet_forecasts = []
prophet_predict_times = {}

print("--- Генерация прогнозов Prophet (на 90 дней) ---")
start_total_predict_time = time.time()

# Создаем датафрейм с будущими датами для ВСЕХ товаров
# Важно, чтобы он содержал все даты тестового периода для каждого item_id
# Используем даты из test_df, чтобы гарантировать совпадение
future_dates_all = test_df[['date', 'item_id']].drop_duplicates().reset_index(drop=True)
# Переименовываем 'date' в 'ds' для Prophet
future_dates_all.rename(columns={'date': 'ds'}, inplace=True)
# Убедимся, что 'ds' имеет правильный тип
future_dates_all['ds'] = pd.to_datetime(future_dates_all['ds'])

print(f"Создан датафрейм для прогнозирования Prophet: {future_dates_all.shape}")


for item_id, model in tqdm(prophet_models.items(), desc="Прогнозирование Prophet"):
    # Создаем заготовку с NaN на случай ошибки
    item_future_dates = future_dates_all[future_dates_all['item_id'] == item_id][['ds']].copy()
    item_forecast_df = item_future_dates.copy()
    item_forecast_df['yhat'] = np.nan
    item_forecast_df['item_id'] = item_id

    if model is not None:
        start_item_predict_time = time.time()
        try:
            # Генерируем прогноз
            item_preds_df = model.predict(item_future_dates[['ds']])
            # Выбираем нужные колонки и мерджим с заготовкой, чтобы сохранить все даты
            item_preds_df = item_preds_df[['ds', 'yhat']]
            # Обновляем yhat в заготовке
            item_forecast_df = pd.merge(item_forecast_df[['ds', 'item_id']], item_preds_df, on='ds', how='left')

            end_item_predict_time = time.time()
            prophet_predict_times[item_id] = end_item_predict_time - start_item_predict_time

        except Exception as e:
            print(f"Ошибка прогнозирования Prophet для {item_id}: {e}. Заполняем NaN.")
            # yhat уже NaN в заготовке
            prophet_predict_times[item_id] = None
    else:
         # yhat уже NaN в заготовке
         prophet_predict_times[item_id] = None

    prophet_forecasts.append(item_forecast_df)


# Объединяем прогнозы
prophet_forecast_df = pd.concat(prophet_forecasts).reset_index(drop=True)
# Переименуем 'yhat' и обработаем NaN/отрицательные значения
prophet_forecast_df.rename(columns={'yhat': 'yhat_prophet'}, inplace=True)
prophet_forecast_df['yhat_prophet'] = prophet_forecast_df['yhat_prophet'].clip(lower=0).fillna(0) # Заменяем NaN на 0
# Переименуем 'ds' обратно в 'date' для слияния
prophet_forecast_df.rename(columns={'ds': 'date'}, inplace=True)


end_total_predict_time = time.time()
print(f"\nОбщее время прогнозирования Prophet: {end_total_predict_time - start_total_predict_time:.2f} сек.")
print(f"Размер датафрейма прогнозов: {prophet_forecast_df.shape}")


# --- Оценка Prophet модели ---
test_merged_prophet = pd.merge(test_df[['date', 'item_id', 'cnt']],
                               prophet_forecast_df,
                               on=['date', 'item_id'],
                               how='left')

# --- Сводная таблица метрик Prophet ---
prophet_results_summary = {}
min_test_date = test_merged_prophet['date'].min()

# Горизонты для оценки
horizons = {'Week': FH_WEEK, 'Month': FH_MONTH, 'Quarter': FH_QUARTER}

# Рассчитываем метрики для каждого горизонта
for name, days in horizons.items():
    # Фильтруем данные для текущего горизонта
    test_horizon = test_merged_prophet[test_merged_prophet['date'] <= min_test_date + pd.Timedelta(days=days-1)]
    # Считаем метрики по каждому товару и усредняем
    horizon_metrics_df = test_horizon.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_prophet'])))
    # Сохраняем средние значения
    prophet_results_summary[name] = horizon_metrics_df.mean()

# Создаем и выводим итоговую таблицу
prophet_summary_df = pd.DataFrame(prophet_results_summary)

print("\n--- Сводная таблица метрик Prophet ---")
print(prophet_summary_df)

# Дополнительно: Вывод метрик по товарам для самого длинного горизонта (квартал)
print("\n--- Метрики Prophet по товарам (Квартал) ---")
prophet_metrics_quarter_df = test_merged_prophet.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_prophet'])))
print(prophet_metrics_quarter_df)

--- Генерация прогнозов Prophet (на 90 дней) ---
Создан датафрейм для прогнозирования Prophet: (1350, 2)


Прогнозирование Prophet:   0%|          | 0/15 [00:00<?, ?it/s]


Общее время прогнозирования Prophet: 1.44 сек.
Размер датафрейма прогнозов: (1350, 3)

--- Сводная таблица метрик Prophet ---
            Week      Month    Quarter
MAE     5.884783   5.265458   6.830777
RMSE    6.649236   6.391558   8.720275
sMAPE  86.743475  84.176485  86.461380
R2     -1.069769  -0.301867  -0.202046

--- Метрики Prophet по товарам (Квартал) ---
                   MAE       RMSE       sMAPE        R2
item_id                                                
STORE_1_064   0.355948   0.532265  174.141063 -0.020356
STORE_1_065   0.762910   1.237892  160.902594 -0.009455
STORE_1_090  40.759729  54.469123   93.108385 -0.421881
STORE_1_252   5.624783   6.711598   39.186467  0.236754
STORE_1_325   3.487476   4.413237   67.288423  0.026197
STORE_1_339   2.339674   3.038566   84.691213 -0.265764
STORE_1_376   0.810543   1.209629  155.404069 -0.311636
STORE_1_546   2.277895   2.754467  120.151307 -0.853244
STORE_1_547  10.524452  12.973974   82.254119 -0.150995
STORE_1_555   6.

In [ ]:
# --- Сохранение моделей ---
MODEL_DIR = "models" # Папка для сохранения моделей
os.makedirs(MODEL_DIR, exist_ok=True) # Создаем папку, если ее нет

# Сохранение ETS моделей
ets_model_path = os.path.join(MODEL_DIR, f"ets_models_{STORE_ID}.joblib")
try:
    joblib.dump(ets_models, ets_model_path)
    print(f"Модели ETS сохранены в: {ets_model_path}")
except Exception as e:
    print(f"Ошибка сохранения моделей ETS: {e}")

# Сохранение Prophet моделей
prophet_model_path = os.path.join(MODEL_DIR, f"prophet_models_{STORE_ID}.joblib")
try:
    # Очистим ненужные компоненты перед сохранением, чтобы уменьшить размер
    for item_id, model in prophet_models.items():
        if model is not None:
            # Удаляем объект stan_fit, он занимает много места и не нужен для predict
            if hasattr(model, 'stan_backend'): # Для новых версий Prophet
               if hasattr(model.stan_backend, 'stan_fit'):
                  del model.stan_backend.stan_fit
            elif hasattr(model, 'stan_fit'): # Для старых версий Prophet
                del model.stan_fit
    joblib.dump(prophet_models, prophet_model_path)
    print(f"Модели Prophet сохранены в: {prophet_model_path}")
except Exception as e:
    print(f"Ошибка сохранения моделей Prophet: {e}")

Модели ETS сохранены в: models/ets_models_STORE_1.joblib
Модели Prophet сохранены в: models/prophet_models_STORE_1.joblib


## Обучение регрессионных моделей

### Обучение и прогнозы TBATS

In [ ]:
# Обучение TBATS




# Словарь для хранения обученных моделей и времени обучения
tbats_models = {}
tbats_fit_times = {}

# Определяем сезонные периоды (недельный, месячный)
# Годовой (365.25) может требовать больше данных и времени
SEASONAL_PERIODS = [7, 30.5, 365.25]

print(f"--- Обучение TBATS (sp={SEASONAL_PERIODS}) для {len(train_df['item_id'].unique())} товаров ---")
start_total_fit_time = time.time()

for item_id in tqdm(train_df['item_id'].unique(), desc="Обучение TBATS"):
    start_item_fit_time = time.time()
    # Готовим данные для sktime
    item_train_series = train_df[train_df['item_id'] == item_id].set_index('date')['cnt'].sort_index()

    # Проверка на достаточность данных (минимум 2 * max(sp))
    min_len = int(2 * max(SEASONAL_PERIODS))
    if len(item_train_series) < min_len:
        print(f"Пропуск {item_id}: недостаточно данных ({len(item_train_series)} точек, нужно >= {min_len})")
        tbats_models[item_id] = None
        tbats_fit_times[item_id] = None
        continue

    # Инициализируем модель TBATS
    # use_arma_errors=False может ускорить обучение, но потенциально снизить точность

    forecaster = TBATS(
        sp=SEASONAL_PERIODS,
        use_box_cox=True,       # Попробуем с Box-Cox
        use_trend=True,         # Позволим моделировать тренд
        use_damped_trend=True,  # Позволим затухающий тренд
        use_arma_errors=False,   # Установим в False для ускорения
        n_jobs=1
    )

    try:
        forecaster.fit(y=item_train_series)
        tbats_models[item_id] = forecaster
        end_item_fit_time = time.time()
        tbats_fit_times[item_id] = end_item_fit_time - start_item_fit_time
    except Exception as e:
        print(f"Ошибка обучения TBATS для {item_id}: {e}")
        tbats_models[item_id] = None
        tbats_fit_times[item_id] = None

end_total_fit_time = time.time()
successful_fits = sum(1 for model in tbats_models.values() if model is not None)
print(f"\nОбучение TBATS завершено.")
print(f"Общее время обучения: {end_total_fit_time - start_total_fit_time:.2f} сек.")
print(f"Успешно обучено моделей: {successful_fits} из {len(train_df['item_id'].unique())}")

valid_fit_times = [t for t in tbats_fit_times.values() if t is not None]
if valid_fit_times:
    print(f"Среднее время обучения на 1 модель: {np.mean(valid_fit_times):.2f} сек.")

--- Обучение TBATS (sp=[7, 30.5, 365.25]) для 15 товаров ---


Обучение TBATS:   0%|          | 0/15 [00:00<?, ?it/s]

/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/mnt/c/Users/edavletshin/courses/!ti


Обучение TBATS завершено.
Общее время обучения: 1286.49 сек.
Успешно обучено моделей: 15 из 15
Среднее время обучения на 1 модель: 85.76 сек.


/mnt/c/Users/edavletshin/courses/!time_series/exam/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [17]:
# Шаг 5.4: Прогнозирование и Оценка TBATS

# Список для хранения прогнозов
tbats_forecasts = []
tbats_predict_times = {}

print("\n--- Генерация прогнозов TBATS (на 90 дней) ---")
start_total_predict_time = time.time()

# Определяем горизонт прогнозирования для sktime
fh = ForecastingHorizon(np.arange(1, FH_QUARTER + 1), is_relative=True)

# Даты тестового периода
test_start_date = test_df['date'].min()
test_end_date = test_df['date'].max()
pred_dates_index = pd.date_range(start=test_start_date, end=test_end_date, freq='D')


for item_id, model in tqdm(tbats_models.items(), desc="Прогнозирование TBATS"):
    # Заготовка DataFrame для товара
    item_forecast_df = pd.DataFrame({'date': pred_dates_index, 'item_id': item_id})

    if model is not None:
        start_item_predict_time = time.time()
        try:
            # Генерируем прогноз
            y_pred = model.predict(fh=fh)
            # Присваиваем правильные даты
            y_pred.index = pred_dates_index
            item_forecast_df['yhat_tbats'] = y_pred.values

            end_item_predict_time = time.time()
            tbats_predict_times[item_id] = end_item_predict_time - start_item_predict_time

        except Exception as e:
            print(f"Ошибка прогнозирования TBATS для {item_id}: {e}")
            item_forecast_df['yhat_tbats'] = np.nan # Заполняем NaN при ошибке
            tbats_predict_times[item_id] = None
    else:
        item_forecast_df['yhat_tbats'] = np.nan # Заполняем NaN, если модель не обучилась
        tbats_predict_times[item_id] = None

    tbats_forecasts.append(item_forecast_df)

# Объединяем прогнозы
tbats_forecast_df = pd.concat(tbats_forecasts).reset_index(drop=True)
# Обрабатываем NaN и отрицательные значения
tbats_forecast_df['yhat_tbats'] = tbats_forecast_df['yhat_tbats'].clip(lower=0).fillna(0)


end_total_predict_time = time.time()
print(f"\nОбщее время прогнозирования TBATS: {end_total_predict_time - start_total_predict_time:.2f} сек.")
print(f"Размер датафрейма прогнозов: {tbats_forecast_df.shape}")


# --- Оценка TBATS модели ---
test_merged_tbats = pd.merge(test_df[['date', 'item_id', 'cnt']],
                               tbats_forecast_df,
                               on=['date', 'item_id'],
                               how='left')

# --- Сводная таблица метрик TBATS ---
tbats_results_summary = {}
min_test_date = test_merged_tbats['date'].min()

# Горизонты для оценки
horizons = {'Week': FH_WEEK, 'Month': FH_MONTH, 'Quarter': FH_QUARTER}

# Рассчитываем метрики для каждого горизонта
for name, days in horizons.items():
    test_horizon = test_merged_tbats[test_merged_tbats['date'] <= min_test_date + pd.Timedelta(days=days-1)]
    horizon_metrics_df = test_horizon.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_tbats'])))
    tbats_results_summary[name] = horizon_metrics_df.mean()

# Создаем и выводим итоговую таблицу
tbats_summary_df = pd.DataFrame(tbats_results_summary)

print("\n--- Сводная таблица метрик TBATS ---")
print(tbats_summary_df)

# Дополнительно: Вывод метрик по товарам для самого длинного горизонта (квартал)
print("\n--- Метрики TBATS по товарам (Квартал) ---")
tbats_metrics_quarter_df = test_merged_tbats.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_tbats'])))
print(tbats_metrics_quarter_df)


--- Генерация прогнозов TBATS (на 90 дней) ---


Прогнозирование TBATS:   0%|          | 0/15 [00:00<?, ?it/s]


Общее время прогнозирования TBATS: 0.28 сек.
Размер датафрейма прогнозов: (1350, 3)

--- Сводная таблица метрик TBATS ---
            Week      Month    Quarter
MAE     4.329738   4.608319   7.241769
RMSE    4.994390   5.734522   8.990226
sMAPE  77.679911  76.456057  84.910888
R2      0.063350   0.012108  -0.369066

--- Метрики TBATS по товарам (Квартал) ---
                   MAE       RMSE       sMAPE        R2
item_id                                                
STORE_1_064   0.338971   0.526847  190.278028  0.000310
STORE_1_065   0.743941   1.261755  168.115741 -0.048748
STORE_1_090  35.960483  44.974450  104.449381  0.030620
STORE_1_252   6.286513   7.771190   41.905530 -0.023264
STORE_1_325   4.301255   5.333301  102.258372 -0.422161
STORE_1_339   2.591551   3.134750   75.478387 -0.347165
STORE_1_376   0.520605   1.119541  103.222161 -0.123541
STORE_1_546   1.987388   2.389154   66.670490 -0.394267
STORE_1_547  14.199131  16.840551   88.396328 -0.939279
STORE_1_555   6.843135

### Обучение и прогнозы ORBIT

In [ ]:
# Подготовка данных и Обучение Orbit (DLT)

# --- Подготовка данных для Orbit ---
orbit_train_df = train_df.copy()
orbit_train_df.rename(columns={'date': 'date_col', 'cnt': 'response_col'}, inplace=True)

# Создаем dummy для event_name_1
event_counts = orbit_train_df['event_name_1'].value_counts()
top_n_events = 10
frequent_events = event_counts.head(top_n_events).index.tolist()
if 'NoEvent' not in frequent_events:
     frequent_events.append('NoEvent')
orbit_train_df['event_name_processed'] = orbit_train_df['event_name_1'].apply(lambda x: x if x in frequent_events else 'OtherEvent')
event_dummies = pd.get_dummies(orbit_train_df['event_name_processed'], prefix='event')
event_dummies.drop(columns=['event_NoEvent'], inplace=True, errors='ignore')

# --- ИСПРАВЛЕНИЕ: Преобразуем dummies в int ---
for col in event_dummies.columns:
    event_dummies[col] = event_dummies[col].astype(int)


orbit_train_df = pd.concat([orbit_train_df, event_dummies], axis=1)

regressor_cols = ['sell_price', 'cashback'] + event_dummies.columns.tolist()
print("Регрессоры для Orbit:", regressor_cols)
# Добавим проверку типов данных перед обучением
print("\nТипы данных регрессоров в orbit_train_df перед обучением:")
print(orbit_train_df[regressor_cols].dtypes)


response_col = 'response_col'
date_col = 'date_col'
panel_col = 'item_id'

# --- Обучение Orbit ---
orbit_models = {}
orbit_fit_times = {}

print(f"\n--- Обучение Orbit DLT (seasonal_period=7) для {orbit_train_df[panel_col].nunique()} товаров ---")
start_total_fit_time = time.time()

unique_items_orbit = orbit_train_df[panel_col].unique()

for item_id in tqdm(unique_items_orbit, desc="Обучение Orbit DLT"):
    start_item_fit_time = time.time()
    item_train_data = orbit_train_df[orbit_train_df[panel_col] == item_id]

    if len(item_train_data) < 14:
        print(f"Пропуск {item_id}: недостаточно данных ({len(item_train_data)} точек)")
        orbit_models[item_id] = None
        orbit_fit_times[item_id] = None
        continue

    # Инициализируем DLT
    dlt = DLT(
        response_col=response_col,
        date_col=date_col,
        regressor_col=regressor_cols,
        seasonality=7,
        seed=42,
        num_warmup=500,
        num_sample=500,
        estimator='stan-map'
    )

    try:
        # Передаем только необходимые колонки с правильными типами
        dlt.fit(df=item_train_data[[date_col, response_col] + regressor_cols])
        orbit_models[item_id] = dlt
        end_item_fit_time = time.time()
        orbit_fit_times[item_id] = end_item_fit_time - start_item_fit_time
    except Exception as e:
        print(f"Ошибка обучения Orbit DLT для {item_id}: {e}")
        # Дополнительно выведем типы данных на момент ошибки
        print(f"Типы данных для {item_id} при ошибке:")
        print(item_train_data[[date_col, response_col] + regressor_cols].dtypes)
        orbit_models[item_id] = None
        orbit_fit_times[item_id] = None


end_total_fit_time = time.time()
successful_fits = sum(1 for model in orbit_models.values() if model is not None)
print(f"\nОбучение Orbit DLT завершено.")
print(f"Общее время обучения: {end_total_fit_time - start_total_fit_time:.2f} сек.")
print(f"Успешно обучено моделей: {successful_fits} из {len(unique_items_orbit)}")

valid_fit_times = [t for t in orbit_fit_times.values() if t is not None]
if valid_fit_times:
    print(f"Среднее время обучения на 1 модель: {np.mean(valid_fit_times):.2f} сек.")

Регрессоры для Orbit: ['sell_price', 'cashback', 'event_ColumbusDay', 'event_Eid al-Fitr', 'event_EidAlAdha', 'event_IndependenceDay', 'event_LaborDay', "event_Mother's day", 'event_NBAFinalsEnd', 'event_OtherEvent', 'event_Ramadan starts', 'event_SuperBowl']

Типы данных регрессоров в orbit_train_df перед обучением:
sell_price               float64
cashback                   int64
event_ColumbusDay          int64
event_Eid al-Fitr          int64
event_EidAlAdha            int64
event_IndependenceDay      int64
event_LaborDay             int64
event_Mother's day         int64
event_NBAFinalsEnd         int64
event_OtherEvent           int64
event_Ramadan starts       int64
event_SuperBowl            int64
dtype: object

--- Обучение Orbit DLT (seasonal_period=7) для 15 товаров ---


Обучение Orbit DLT:   0%|          | 0/15 [00:00<?, ?it/s]

2025-04-26 23:12:14 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025-04-26 23:12:17 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025-04-26 23:12:18 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025-04-26 23:12:19 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025-04-26 23:12:19 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025-04-26 23:12:19 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025-04-26 23:12:20 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025-04-26 23:12:20 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025-04-26 23:12:20 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025-04-26 23:12:21 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025-04-26 23:12:21 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025-04-26 23:12:21 - orbit - INFO - Optimizing (CmdStanPy) with algorithm: LBFGS.
2025


Обучение Orbit DLT завершено.
Общее время обучения: 8.78 сек.
Успешно обучено моделей: 15 из 15
Среднее время обучения на 1 модель: 0.58 сек.


In [ ]:
# Подготовка данных для прогноза и Оценка Orbit (DLT)

orbit_forecasts = []
orbit_predict_times = {}

print("\n--- Генерация прогнозов Orbit DLT (на 90 дней) ---")
start_total_predict_time = time.time()

# --- Создание future_df для прогнозирования ---
test_start_date = test_df['date'].min()
test_end_date = test_df['date'].max()
pred_dates_index = pd.date_range(start=test_start_date, end=test_end_date, freq='D')

future_df_list = []
for item_id in unique_items_orbit: # Используем unique_items_orbit из ячейки 5.5
    future_df_list.append(pd.DataFrame({
        'date_col': pred_dates_index,
        panel_col: item_id # panel_col = 'item_id'
    }))
future_df_full = pd.concat(future_df_list).reset_index(drop=True)

# Добавляем будущие значения регрессоров
# Цена
last_prices = train_df.loc[train_df.groupby('item_id')['date'].idxmax()][['item_id', 'sell_price']]
future_df_full = pd.merge(future_df_full, last_prices, on=panel_col, how='left')
# Кэшбэк
last_cashback = train_df.loc[train_df.groupby('item_id')['date'].idxmax()][['item_id', 'cashback']]
future_df_full = pd.merge(future_df_full, last_cashback, on=panel_col, how='left')

# Праздники/События (event dummies)

calendar_full_df = pd.read_csv('data/shop_sales_dates.csv', parse_dates=['date'])
calendar_full_df['event_name_1'].fillna('NoEvent', inplace=True)
# Используем те же frequent_events, что и при обучении
calendar_full_df['event_name_processed'] = calendar_full_df['event_name_1'].apply(lambda x: x if x in frequent_events else 'OtherEvent')
event_dummies_future_cal = pd.get_dummies(calendar_full_df['event_name_processed'], prefix='event')
event_dummies_future_cal.drop(columns=['event_NoEvent'], inplace=True, errors='ignore')
# --- Преобразуем dummies в int ---
for col in event_dummies_future_cal.columns:
    event_dummies_future_cal[col] = event_dummies_future_cal[col].astype(int)

calendar_subset_df = pd.concat([calendar_full_df[['date']], event_dummies_future_cal], axis=1)
calendar_subset_df.rename(columns={'date': 'date_col'}, inplace=True)


future_df_full = pd.merge(future_df_full, calendar_subset_df, on='date_col', how='left')
# Убедимся, что ВСЕ dummy колонки существуют и заполняем NaN нулями, приводим к int
for col in event_dummies.columns: # Используем колонки из event_dummies, созданных при обучении
     if col not in future_df_full.columns:
           future_df_full[col] = 0 # Добавляем недостающие колонки
     future_df_full[col] = future_df_full[col].fillna(0).astype(int)


print("Future DF подготовлен. Пример:")
print(future_df_full.head())
print("\nТипы данных регрессоров в future_df_full:")
print(future_df_full[regressor_cols].dtypes) # Проверяем типы
print("\nПроверка NaN в регрессорах Future DF:")
print(future_df_full[regressor_cols].isnull().sum()) # Должны быть нули


# --- Прогнозирование ---
for item_id, model in tqdm(orbit_models.items(), desc="Прогнозирование Orbit DLT"):
    item_forecast_df = pd.DataFrame({'date': pred_dates_index, 'item_id': item_id})
    if model is not None:
        start_item_predict_time = time.time()
        try:
            item_future_df = future_df_full[future_df_full[panel_col] == item_id]
            # Убедимся, что передаем нужные колонки
            predicted_df = model.predict(df=item_future_df[[date_col] + regressor_cols], decompose=False)
            predicted_df = predicted_df[['date_col', 'prediction']].rename(columns={'date_col': 'date', 'prediction': 'yhat_orbit'})
            item_forecast_df = pd.merge(item_forecast_df[['date', 'item_id']], predicted_df, on='date', how='left')
            end_item_predict_time = time.time()
            orbit_predict_times[item_id] = end_item_predict_time - start_item_predict_time
        except Exception as e:
            print(f"Ошибка прогнозирования Orbit DLT для {item_id}: {e}")
            item_forecast_df['yhat_orbit'] = np.nan
            orbit_predict_times[item_id] = None
    else:
        item_forecast_df['yhat_orbit'] = np.nan
        orbit_predict_times[item_id] = None
    orbit_forecasts.append(item_forecast_df)

orbit_forecast_df = pd.concat(orbit_forecasts).reset_index(drop=True)
orbit_forecast_df['yhat_orbit'] = orbit_forecast_df['yhat_orbit'].clip(lower=0).fillna(0)
end_total_predict_time = time.time()
print(f"\nОбщее время прогнозирования Orbit DLT: {end_total_predict_time - start_total_predict_time:.2f} сек.")
print(f"Размер датафрейма прогнозов: {orbit_forecast_df.shape}")



--- Генерация прогнозов Orbit DLT (на 90 дней) ---
Future DF подготовлен. Пример:
    date_col      item_id  sell_price  cashback  event_ColumbusDay  event_Eid al-Fitr  event_EidAlAdha  event_IndependenceDay  event_LaborDay  event_Mother's day  event_NBAFinalsEnd  event_OtherEvent  event_Ramadan starts  event_SuperBowl
0 2015-10-24  STORE_1_064        2.68         0                  0                  0                0                      0               0                   0                   0                 0                     0                0
1 2015-10-25  STORE_1_064        2.68         0                  0                  0                0                      0               0                   0                   0                 0                     0                0
2 2015-10-26  STORE_1_064        2.68         0                  0                  0                0                      0               0                   0                   0                 0 

/tmp/ipykernel_5233/1531487694.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  calendar_full_df['event_name_1'].fillna('NoEvent', inplace=True)


Прогнозирование Orbit DLT:   0%|          | 0/15 [00:00<?, ?it/s]


Общее время прогнозирования Orbit DLT: 0.71 сек.
Размер датафрейма прогнозов: (1350, 3)


In [74]:
# --- Оценка Orbit DLT модели ---
test_merged_orbit = pd.merge(test_df[['date', 'item_id', 'cnt']],
                             orbit_forecast_df,
                             on=['date', 'item_id'],
                             how='left')

# --- Сводная таблица метрик Orbit DLT ---
orbit_results_summary = {}
min_test_date = test_merged_orbit['date'].min()
horizons = {'Week': FH_WEEK, 'Month': FH_MONTH, 'Quarter': FH_QUARTER}
for name, days in horizons.items():
    test_horizon = test_merged_orbit[test_merged_orbit['date'] <= min_test_date + pd.Timedelta(days=days-1)]
    horizon_metrics_df = test_horizon.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_orbit'])))
    orbit_results_summary[name] = horizon_metrics_df.mean()
orbit_summary_df = pd.DataFrame(orbit_results_summary)
print("\n--- Сводная таблица метрик Orbit DLT ---")
print(orbit_summary_df)

# Дополнительно: Метрики по товарам (Квартал)
print("\n--- Метрики Orbit DLT по товарам (Квартал) ---")
orbit_metrics_quarter_df = test_merged_orbit.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_orbit'])))
print(orbit_metrics_quarter_df)


--- Сводная таблица метрик Orbit DLT ---
            Week      Month    Quarter
MAE     4.063869   4.625516   6.897581
RMSE    4.828499   5.754251   8.906294
sMAPE  79.971416  84.785811  95.731245
R2      0.121218  -0.023963  -0.322541

--- Метрики Orbit DLT по товарам (Квартал) ---
                   MAE       RMSE       sMAPE        R2
item_id                                                
STORE_1_064   0.240415   0.546027  168.704188 -0.073799
STORE_1_065   0.676795   1.342668  189.000317 -0.187569
STORE_1_090  41.019722  51.027605   95.948976 -0.247880
STORE_1_252   5.111924   6.954684   35.793986  0.180465
STORE_1_325   4.439004   5.612750  111.376188 -0.575099
STORE_1_339   3.010312   3.895439  138.034179 -1.080310
STORE_1_376   0.482540   1.138731   71.747603 -0.162389
STORE_1_546   2.252941   2.729819  114.441779 -0.820224
STORE_1_547  14.110060  18.588138  169.765857 -1.362650
STORE_1_555   6.373809   7.820595   29.012360  0.084094
STORE_1_584   1.453232   2.553474  132.2242

### Промежуточные метрики

In [ ]:
# --- Создание сводных таблиц сравнения ВСЕХ моделей по горизонтам ---

# Список моделей и их сводных таблиц
model_summaries = {
    'Baseline': 'baseline_summary_df',
    'AutoARIMA': 'autoarima_summary_df',
    'ETS': 'ets_summary_df',
    'Prophet': 'prophet_summary_df',
    'TBATS': 'tbats_summary_df',
    'Orbit': 'orbit_summary_df'
}

# Проверяем наличие всех необходимых DataFrame'ов
missing_dfs = []
for df_name in model_summaries.values():
    if df_name not in locals(): # locals() содержит локальные переменные
        missing_dfs.append(df_name)

if missing_dfs:
    print(f"Ошибка: Не найдены следующие DataFrame'ы с результатами: {', '.join(missing_dfs)}")
    print("Пожалуйста, убедитесь, что код оценки для ВСЕХ моделей был выполнен.")
else:
    # --- Таблица для Недельного горизонта (FH=7) ---
    week_metrics_list = {}
    for model_name, df_name in model_summaries.items():
        df = locals()[df_name] # Получаем DataFrame по имени
        if 'Week' in df.columns:
             week_metrics_list[model_name] = df['Week']
        else:
             print(f"Предупреждение: Колонка 'Week' не найдена в {df_name}")

    metrics_week_comparison = pd.concat(week_metrics_list, axis=1)
    print("\n--- Сравнение моделей: Горизонт - Неделя ---")
    print(metrics_week_comparison.round(3))


    # --- Таблица для Месячного горизонта (FH=30) ---
    month_metrics_list = {}
    for model_name, df_name in model_summaries.items():
        df = locals()[df_name]
        if 'Month' in df.columns:
            month_metrics_list[model_name] = df['Month']
        else:
            print(f"Предупреждение: Колонка 'Month' не найдена в {df_name}")

    metrics_month_comparison = pd.concat(month_metrics_list, axis=1)
    print("\n--- Сравнение моделей: Горизонт - Месяц ---")
    print(metrics_month_comparison.round(3))


    # --- Таблица для Квартального горизонта (FH=90) ---
    quarter_metrics_list = {}
    for model_name, df_name in model_summaries.items():
        df = locals()[df_name]
        if 'Quarter' in df.columns:
            quarter_metrics_list[model_name] = df['Quarter']
        else:
             print(f"Предупреждение: Колонка 'Quarter' не найдена в {df_name}")

    metrics_quarter_comparison = pd.concat(quarter_metrics_list, axis=1)
    print("\n--- Сравнение моделей: Горизонт - Квартал ---")
    print(metrics_quarter_comparison.round(3))


--- Сравнение моделей: Горизонт - Неделя ---
       Baseline  AutoARIMA     ETS  Prophet   TBATS   Orbit
MAE       5.181      4.621   4.149    5.885   4.330   4.064
RMSE      6.515      5.298   4.872    6.649   4.994   4.828
sMAPE    69.216     79.628  76.182   86.743  77.680  79.971
R2       -0.820     -0.020   0.098   -1.070   0.063   0.113

--- Сравнение моделей: Горизонт - Месяц ---
       Baseline  AutoARIMA     ETS  Prophet   TBATS   Orbit
MAE       5.696      4.911   4.785    5.265   4.608   4.626
RMSE      7.372      5.839   5.833    6.392   5.735   5.754
sMAPE    70.952     81.026  79.566   84.176  76.456  84.786
R2       -0.687      0.035   0.076   -0.302   0.012  -0.024

--- Сравнение моделей: Горизонт - Квартал ---
       Baseline  AutoARIMA     ETS  Prophet   TBATS   Orbit
MAE       7.934      7.141   7.284    6.831   7.242   6.898
RMSE     10.240      9.064   9.310    8.720   8.990   8.906
sMAPE    78.637     85.592  86.067   86.461  84.911  95.731
R2       -0.658     -0

In [ ]:
# --- Создание СОРТИРОВАННЫХ сводных таблиц сравнения моделей по горизонтам (по MAE) ---

# Проверяем их наличие еще раз на всякий случай
comparison_dfs_exist = True
for df_name in ['metrics_week_comparison', 'metrics_month_comparison', 'metrics_quarter_comparison']:
    if df_name not in locals():
        print(f"Ошибка: Не найден DataFrame {df_name}. Пожалуйста, выполните предыдущий шаг.")
        comparison_dfs_exist = False
        break

if comparison_dfs_exist:
    # --- Сортированная таблица для Недельного горизонта (FH=7) ---
    metrics_week_sorted = metrics_week_comparison.sort_values(by='MAE', axis=1) # Сортируем КОЛОНКИ по строке MAE

    print("\n--- Сравнение моделей: Горизонт - Неделя (Отсортировано по MAE) ---")
    print(metrics_week_sorted.round(3))


    # --- Сортированная таблица для Месячного горизонта (FH=30) ---
    metrics_month_sorted = metrics_month_comparison.sort_values(by='MAE', axis=1)

    print("\n--- Сравнение моделей: Горизонт - Месяц (Отсортировано по MAE) ---")
    print(metrics_month_sorted.round(3))


    # --- Сортированная таблица для Квартального горизонта (FH=90) ---
    metrics_quarter_sorted = metrics_quarter_comparison.sort_values(by='MAE', axis=1)

    print("\n--- Сравнение моделей: Горизонт - Квартал (Отсортировано по MAE) ---")
    print(metrics_quarter_sorted.round(3))


--- Сравнение моделей: Горизонт - Неделя (Отсортировано по MAE) ---
        Orbit     ETS   TBATS  AutoARIMA  Baseline  Prophet
MAE     4.064   4.149   4.330      4.621     5.181    5.885
RMSE    4.828   4.872   4.994      5.298     6.515    6.649
sMAPE  79.971  76.182  77.680     79.628    69.216   86.743
R2      0.113   0.098   0.063     -0.020    -0.820   -1.070

--- Сравнение моделей: Горизонт - Месяц (Отсортировано по MAE) ---
        TBATS   Orbit     ETS  AutoARIMA  Prophet  Baseline
MAE     4.608   4.626   4.785      4.911    5.265     5.696
RMSE    5.735   5.754   5.833      5.839    6.392     7.372
sMAPE  76.456  84.786  79.566     81.026   84.176    70.952
R2      0.012  -0.024   0.076      0.035   -0.302    -0.687

--- Сравнение моделей: Горизонт - Квартал (Отсортировано по MAE) ---
       Prophet   Orbit  AutoARIMA   TBATS     ETS  Baseline
MAE      6.831   6.898      7.141   7.242   7.284     7.934
RMSE     8.720   8.906      9.064   8.990   9.310    10.240
sMAPE   86.46

## LightGBM

### Создание признаков и первое обучение

In [ ]:
def create_features(df, target_col='cnt'):
    """Создает признаки для модели LightGBM из датафрейма."""
    df = df.copy()
    # Убедимся, что 'date' точно есть и имеет правильный тип
    if 'date' not in df.columns:
        raise KeyError("Столбец 'date' отсутствует в DataFrame, передаваемом в create_features.")
    df['date'] = pd.to_datetime(df['date'])

    # 1. Календарные признаки
    df['dayofweek'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.isocalendar().week.astype(int)
    df['dayofyear'] = df['date'].dt.dayofyear

    # 2. Лаговые признаки продаж (cnt)
    lags = [7, 14, 21, 28, 35, 90]
    df = df.sort_values(by=['item_id', 'date'])
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df.groupby('item_id')[target_col].shift(lag)

    # 3. Скользящие оконные признаки продаж (cnt)
    windows = [7, 14, 28]
    # Сдвиг на 7 дней, чтобы использовать данные до начала текущей недели
    base_shift = 7
    for window in windows:
        df[f'{target_col}_roll_mean_{window}'] = df.groupby('item_id')[target_col].shift(base_shift).rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df.groupby('item_id')[target_col].shift(base_shift).rolling(window).std()

    # 4. Праздники/События (dummy для event_name_1)
    df['event_name_processed'] = df['event_name_1'].apply(lambda x: x if x in frequent_events else 'OtherEvent')
    event_dummies = pd.get_dummies(df['event_name_processed'], prefix='event_name') # Изменил префикс для ясности
    event_dummies.drop(columns=['event_name_NoEvent'], inplace=True, errors='ignore')
    for col in event_dummies.columns:
         event_dummies[col] = event_dummies[col].astype(int)
    df = pd.concat([df, event_dummies], axis=1)

    # 4.1. Кодирование event_type_1 (dummy)
    event_type_dummies = pd.get_dummies(df['event_type_1'], prefix='event_type')
    event_type_dummies.drop(columns=['event_type_NoType'], inplace=True, errors='ignore') # Удаляем базовую категорию
    for col in event_type_dummies.columns:
        event_type_dummies[col] = event_type_dummies[col].astype(int)
    df = pd.concat([df, event_type_dummies], axis=1)

    # 5. ID товара как категориальный признак
    # Создаем и сохраняем encoder для возможности обратного преобразования
    item_encoder = LabelEncoder()
    df['item_id_encoded'] = item_encoder.fit_transform(df['item_id'])


    # Удаляем ненужные колонки
    cols_to_drop = ['date_id', 'wm_yr_wk', 'weekday', 'event_name_1', 'event_name_processed', 'item_id', 'event_type_1']
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
    df.head()

    return df, item_encoder # Возвращаем encoder вместе с df

# --- Применяем функцию для создания признаков ---
print("Создание признаков для LGBM...")

combined_df_for_features = pd.concat([train_df, test_df], ignore_index=True)
print(f"Объединенный датафрейм для создания признаков: {combined_df_for_features.shape}")
print("Проверка столбцов в combined_df_for_features:")
print(combined_df_for_features.columns) # Убедимся, что 'date' на месте

full_df_features, item_le = create_features(combined_df_for_features) # Теперь передаем объединенный df



# --- Разделяем на Train/Test уже с признаками ---
# Важно использовать ту же самую split_date, что и раньше
train_features_df = full_df_features[full_df_features['date'] < split_date].copy()
test_features_df = full_df_features[full_df_features['date'] >= split_date].copy()

# Удаляем строки с NaN из ТРЕНИРОВОЧНОЙ выборки
train_initial_rows = len(train_features_df)
train_features_df.dropna(inplace=True)
print(f"Удалено {train_initial_rows - len(train_features_df)} строк с NaN из train_features_df.")
print(f"Размер train_features_df после удаления NaN: {train_features_df.shape}")
print(f"Размер test_features_df: {test_features_df.shape}")

# Проверим NaN в тестовой выборке (не должно быть, если лаги < TEST_SIZE)
print(f"Количество NaN в test_features_df: {test_features_df.isnull().sum().sum()}")


# --- Определяем признаки (X) и таргет (y) ---
TARGET = 'cnt'
FEATURES = [col for col in train_features_df.columns if col not in [TARGET, 'date']]
# Категориальные признаки для LightGBM
CATEGORICAL_FEATURES = ['item_id_encoded', 'dayofweek', 'month', 'year', 'dayofmonth', 'weekofyear', 'dayofyear']
event_name_dummies = [col for col in FEATURES if col.startswith('event_name_')] # Dummies от event_name
event_type_dummies = [col for col in FEATURES if col.startswith('event_type_')] # Dummies от event_type
CATEGORICAL_FEATURES.extend(event_name_dummies)
CATEGORICAL_FEATURES.extend(event_type_dummies)
CATEGORICAL_FEATURES = [col for col in CATEGORICAL_FEATURES if col in FEATURES] # Перепроверка


X_train = train_features_df[FEATURES]
y_train = train_features_df[TARGET]
X_test = test_features_df[FEATURES]
y_test = test_features_df[TARGET]

print(f"\nКоличество признаков: {len(FEATURES)}")
print("Примеры признаков:", FEATURES[:5], "...", FEATURES[-5:])
print("Категориальные признаки:", CATEGORICAL_FEATURES)
# Сохраняем item_le для использования в оценке
# Можно сделать его глобальным или передать в функцию оценки
global saved_item_encoder
saved_item_encoder = item_le

Создание признаков для LGBM...
Объединенный датафрейм для создания признаков: (27285, 13)
Проверка столбцов в combined_df_for_features:
Index(['item_id', 'date_id', 'cnt', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'cashback', 'sell_price'], dtype='object')
Удалено 1350 строк с NaN из train_features_df.
Размер train_features_df после удаления NaN: (24585, 38)
Размер test_features_df: (1350, 38)
Количество NaN в test_features_df: 0

Количество признаков: 36
Примеры признаков: ['wday', 'month', 'year', 'cashback', 'sell_price'] ... ['event_type_Cultural', 'event_type_National', 'event_type_Religious', 'event_type_Sporting', 'item_id_encoded']
Категориальные признаки: ['item_id_encoded', 'dayofweek', 'month', 'year', 'dayofmonth', 'weekofyear', 'dayofyear', 'event_name_ColumbusDay', 'event_name_Eid al-Fitr', 'event_name_EidAlAdha', 'event_name_IndependenceDay', 'event_name_LaborDay', "event_name_Mother's day", 'event_name_NBAFinalsEnd', 'event_

In [41]:
# Шаг 6.2: Обучение, Прогнозирование и Оценка LightGBM

print("\n--- Обучение LightGBM ---")
start_fit_time = time.time()

# Параметры LightGBM
params = {
    'objective': 'regression_l1', # MAE loss
    'metric': 'mae',
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'num_leaves': 31,
    'verbose': -1,
    'n_jobs': -1,
    'seed': 42,
    'boosting_type': 'gbdt',
}

model_lgb = lgb.LGBMRegressor(**params)

# Обучение модели с early stopping на тестовом наборе
# (Помним про потенциальную "утечку" информации при использовании теста для early stopping)
print("Обучение модели LGBM...")
model_lgb.fit(X_train, y_train,
              eval_set=[(X_test, y_test)],
              eval_metric='mae',
              callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)],
              categorical_feature=CATEGORICAL_FEATURES) # Передаем категориальные признаки

end_fit_time = time.time()
print(f"Обучение LightGBM завершено за {end_fit_time - start_fit_time:.2f} сек.")
if model_lgb.best_iteration_ is not None:
    print(f"Лучшая итерация: {model_lgb.best_iteration_}")
else:
    print("Ранняя остановка не сработала (возможно, все n_estimators использованы).")


# --- Прогнозирование LightGBM ---
print("\n--- Генерация прогнозов LightGBM ---")
start_predict_time = time.time()

# Используем лучшую итерацию, если она есть, иначе все деревья
best_iter = model_lgb.best_iteration_ if model_lgb.best_iteration_ is not None else params['n_estimators']
y_pred_lgb = model_lgb.predict(X_test, num_iteration=best_iter)
# Обрезаем отрицательные значения
y_pred_lgb = np.maximum(0, y_pred_lgb)

end_predict_time = time.time()
print(f"Прогнозирование LightGBM завершено за {end_predict_time - start_predict_time:.2f} сек.")


# --- Оценка LightGBM модели ---
# Используем сохраненный LabelEncoder для декодирования item_id
try:
    # Проверяем, доступен ли encoder, сохраненный в предыдущей ячейке
    decoder_map = dict(zip(saved_item_encoder.transform(saved_item_encoder.classes_), saved_item_encoder.classes_))
except NameError:
    print("Ошибка: LabelEncoder для item_id (saved_item_encoder) не найден.")
    # Попытка восстановить, если train_df все еще доступен
    try:
        item_le_fallback = LabelEncoder().fit(train_df['item_id']) # train_df до создания признаков
        decoder_map = dict(zip(item_le_fallback.transform(item_le_fallback.classes_), item_le_fallback.classes_))
        print("Используется восстановленный LabelEncoder.")
    except NameError:
        print("Не удалось восстановить LabelEncoder. Декодирование item_id невозможно.")
        decoder_map = {} # Оставляем пустым, оценка по товарам не будет работать корректно


# Создаем DataFrame с результатами для оценки по горизонтам
lgbm_results_df = pd.DataFrame({
    'date': test_features_df['date'].values, # Используем .values для избежания проблем с индексом
    'item_id': test_features_df['item_id_encoded'].map(decoder_map), # Декодируем item_id
    'cnt': y_test.values,
    'yhat_lgbm': y_pred_lgb
})
# Проверим, удалось ли декодирование
if lgbm_results_df['item_id'].isnull().any():
    print("Предупреждение: Не удалось декодировать некоторые item_id_encoded.")


# --- Сводная таблица метрик LightGBM ---
lgbm_results_summary = {}
min_test_date = lgbm_results_df['date'].min()

horizons = {'Week': FH_WEEK, 'Month': FH_MONTH, 'Quarter': FH_QUARTER}
print("\nРасчет метрик для LightGBM...")
for name, days in horizons.items():
    test_horizon = lgbm_results_df[lgbm_results_df['date'] <= min_test_date + pd.Timedelta(days=days-1)]
    if test_horizon.empty or test_horizon['item_id'].isnull().all(): # Добавлена проверка на null в item_id
        print(f"Предупреждение: Нет данных или не удалось декодировать item_id для горизонта '{name}'.")
        lgbm_results_summary[name] = pd.Series({'MAE': np.nan, 'RMSE': np.nan, 'sMAPE': np.nan, 'R2': np.nan})
        continue
    # Считаем метрики по каждому товару и усредняем
    # Добавим обработку ошибок на случай проблем с декодированием или пустыми группами
    try:
        horizon_metrics_df = test_horizon.groupby('item_id').filter(lambda x: not x.empty).groupby('item_id')\
                                     .apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_lgbm'])))
        if not horizon_metrics_df.empty:
             lgbm_results_summary[name] = horizon_metrics_df.mean()
        else:
             print(f"Предупреждение: Нет валидных групп для расчета метрик горизонта '{name}'.")
             lgbm_results_summary[name] = pd.Series({'MAE': np.nan, 'RMSE': np.nan, 'sMAPE': np.nan, 'R2': np.nan})

    except Exception as e:
        print(f"Ошибка при расчете метрик для горизонта '{name}': {e}")
        lgbm_results_summary[name] = pd.Series({'MAE': np.nan, 'RMSE': np.nan, 'sMAPE': np.nan, 'R2': np.nan})


# Создаем и выводим итоговую таблицу
lgbm_summary_df = pd.DataFrame(lgbm_results_summary)
print("\n--- Сводная таблица метрик LightGBM ---")
print(lgbm_summary_df)


# Дополнительно: Метрики по товарам (Квартал)
print("\n--- Метрики LightGBM по товарам (Квартал) ---")
if not lgbm_results_df.empty and not lgbm_results_df['item_id'].isnull().all():
    try:
        lgbm_metrics_quarter_df = lgbm_results_df.groupby('item_id').filter(lambda x: not x.empty).groupby('item_id')\
                                             .apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_lgbm'])))
        print(lgbm_metrics_quarter_df)
    except Exception as e:
         print(f"Ошибка при расчете метрик по товарам: {e}")

else:
    print("DataFrame lgbm_results_df пуст или не содержит валидных item_id, метрики по товарам не рассчитаны.")


# Дополнительно: Важность признаков
try:
    print("\n--- Важность признаков LightGBM ---")
    feature_importance_df = pd.DataFrame({
        'feature': model_lgb.feature_name_,
        'importance': model_lgb.feature_importances_
    }).sort_values(by='importance', ascending=False)
    print(feature_importance_df)
except Exception as e:
    print(f"Не удалось получить важность признаков: {e}")


--- Обучение LightGBM ---
Обучение модели LGBM...
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
Обучение LightGBM завершено за 9.56 сек.
Лучшая итерация: 114

--- Генерация прогнозов LightGBM ---
Прогнозирование LightGBM завершено за 0.01 сек.

Расчет метрик для LightGBM...

--- Сводная таблица метрик LightGBM ---
            Week      Month    Quarter
MAE     4.122629   4.300051   5.143933
RMSE    5.157964   5.400141   7.137492
sMAPE  81.990873  81.278413  84.822289
R2     -0.059663   0.036919   0.050941

--- Метрики LightGBM по товарам (Квартал) ---
                   MAE       RMSE       sMAPE        R2
item_id                                                
STORE_1_064   0.237064   0.563642  185.406266 -0.144201
STORE_1_065   0.785532   1.395536  172.288577 -0.282931
STORE_1_090  24.293486  37.991931  100.645914  0.308256
STORE_1_252   4.977374   6.427280   35.524721  0.300050
STORE_1_325  

### Упрощенные признаки, повторное обучение

In [ ]:
def create_features_simplified(df, target_col='cnt'):
    """Создает УПРОЩЕННЫЙ набор признаков для модели LightGBM."""
    df = df.copy()
    if 'date' not in df.columns: raise KeyError("Столбец 'date' отсутствует.")
    df['date'] = pd.to_datetime(df['date'])

    # 1. ВАЖНЫЕ Календарные признаки
    df['dayofweek'] = df['date'].dt.dayofweek
    df['year'] = df['date'].dt.year
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.isocalendar().week.astype(int)
    df['dayofyear'] = df['date'].dt.dayofyear
    # wday остается, если был на входе

    # 2. ВАЖНЫЕ Лаговые признаки продаж
    lags = [7, 14, 21, 28, 35, 90]
    df = df.sort_values(by=['item_id', 'date'])
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df.groupby('item_id')[target_col].shift(lag)

    # 3. ВАЖНЫЕ Скользящие оконные признаки
    windows = [7, 14, 28]
    base_shift = 7
    for window in windows:
        shifted = df.groupby('item_id')[target_col].shift(base_shift)
        df[f'{target_col}_roll_mean_{window}'] = shifted.rolling(window, min_periods=1).mean()
        df[f'{target_col}_roll_std_{window}'] = shifted.rolling(window, min_periods=2).std()


    # --- 5. УПРОЩЕННОЕ Кодирование ID товара (Жесткий маппинг) ---
    # Словарь: ключ - исходный ID, значение - код 0-14
    item_id_map = {
        'STORE_1_064': 0, 'STORE_1_065': 1, 'STORE_1_090': 2, 'STORE_1_252': 3,
        'STORE_1_325': 4, 'STORE_1_339': 5, 'STORE_1_376': 6, 'STORE_1_546': 7,
        'STORE_1_547': 8, 'STORE_1_555': 9, 'STORE_1_584': 10, 'STORE_1_586': 11,
        'STORE_1_587': 12, 'STORE_1_714': 13, 'STORE_1_727': 14
    }
    # Применяем маппинг, неизвестные ID получат NaN (потом заменим на -1 или 0)
    df['item_id_encoded'] = df['item_id'].map(item_id_map)
    # Заполняем пропуски (если вдруг появятся не те ID) значением -1
    df['item_id_encoded'] = df['item_id_encoded'].fillna(-1)
    # Преобразуем в целочисленный тип
    df['item_id_encoded'] = df['item_id_encoded'].astype('int16') # int16 достаточно для 0-14


    # 7. Удаляем ненужные столбцы
    cols_to_drop = [
        'date_id', 'wm_yr_wk', 'weekday', # Ненужные календарные/id
        'event_name_1', 'event_type_1',    # Ненужные события
        'item_id',                         # Исходный ID товара
        'cashback'                         # Ненужный кэшбэк (если был)
    ]
    # Проверяем наличие столбцов перед удалением
    cols_to_drop_existing = [col for col in cols_to_drop if col in df.columns]
    df.drop(columns=cols_to_drop_existing, inplace=True, errors='ignore')

    return df

# --- Применяем УПРОЩЕННУЮ функцию для создания признаков ---
print("Создание УПРОЩЕННЫХ признаков для LGBM (с хардкодом ID)...")

# Объединяем train и test
combined_df_for_features = pd.concat([train_df, test_df], ignore_index=True)
print(f"Объединенный датафрейм: {combined_df_for_features.shape}")


full_df_features_simplified = create_features_simplified(combined_df_for_features)

# --- Разделяем на Train/Test ---
train_features_df = full_df_features_simplified[full_df_features_simplified['date'] < split_date].copy()
test_features_df = full_df_features_simplified[full_df_features_simplified['date'] >= split_date].copy()

# Удаляем строки с NaN из Train
train_initial_rows = len(train_features_df); train_features_df.dropna(inplace=True)
print(f"Удалено {train_initial_rows - len(train_features_df)} строк с NaN из train_features_df.")
print(f"Размер train_features_df: {train_features_df.shape}")
print(f"Размер test_features_df: {test_features_df.shape}")
print(f"Количество NaN в test_features_df: {test_features_df.isnull().sum().sum()}")


# --- Определяем признаки (X) и таргет (y) ---
TARGET = 'cnt'
FEATURES = [col for col in train_features_df.columns if col not in [TARGET, 'date']]
CATEGORICAL_FEATURES = [
    'item_id_encoded',
    'dayofweek', 'year', 'dayofmonth',
    'weekofyear', 'dayofyear', 'wday'
]
CATEGORICAL_FEATURES = [col for col in CATEGORICAL_FEATURES if col in FEATURES]



X_train = train_features_df[FEATURES]
y_train = train_features_df[TARGET]
X_test = test_features_df[FEATURES]
y_test = test_features_df[TARGET]

print(f"\nКоличество признаков: {len(FEATURES)}")
print("Первые 5 признаков:", FEATURES[:5])
print("Последние 5 признаков:", FEATURES[-5:])
print("Категориальные признаки:", CATEGORICAL_FEATURES)

Создание УПРОЩЕННЫХ признаков для LGBM (с хардкодом ID)...
Объединенный датафрейм: (27285, 13)
Удалено 1350 строк с NaN из train_features_df.
Размер train_features_df: (24585, 23)
Размер test_features_df: (1350, 23)
Количество NaN в test_features_df: 0

Количество признаков: 21
Первые 5 признаков: ['wday', 'month', 'year', 'sell_price', 'dayofweek']
Последние 5 признаков: ['cnt_roll_mean_14', 'cnt_roll_std_14', 'cnt_roll_mean_28', 'cnt_roll_std_28', 'item_id_encoded']
Категориальные признаки: ['item_id_encoded', 'dayofweek', 'year', 'dayofmonth', 'weekofyear', 'dayofyear', 'wday']


In [ ]:
# Обучение, Прогнозирование, Оценка - Упрощенная версия

print("\n--- Обучение LightGBM (упрощенные признаки) ---")
start_fit_time = time.time()

# Параметры LightGBM
params = {
    'objective': 'regression_l1', 'metric': 'mae', 'n_estimators': 57,
    'learning_rate': 0.1, 'num_leaves': 31, 'feature_fraction': 0.8,
    'bagging_fraction': 0.9, 'bagging_freq': 1, 'verbose': -1,
    'n_jobs': -1, 'seed': 42, 'boosting_type': 'gbdt'
    # Убрали n_estimators=2000 и early_stopping для финальной модели
}

# Обучаем финальную модель с фиксированными параметрами
model_lgb_simplified = lgb.LGBMRegressor(**params)
print("Обучение финальной модели LGBM...")
model_lgb_simplified.fit(X_train, y_train, categorical_feature=CATEGORICAL_FEATURES)


end_fit_time = time.time()
print(f"Обучение LightGBM завершено за {end_fit_time - start_fit_time:.2f} сек.")

# --- Оценка LightGBM модели ---
# --- Создаем обратный маппинг для item_id ---
item_id_map_reverse = { # Обратный словарь к тому, что в create_features
    0: 'STORE_1_064', 1: 'STORE_1_065', 2: 'STORE_1_090', 3: 'STORE_1_252',
    4: 'STORE_1_325', 5: 'STORE_1_339', 6: 'STORE_1_376', 7: 'STORE_1_546',
    8: 'STORE_1_547', 9: 'STORE_1_555', 10: 'STORE_1_584', 11: 'STORE_1_586',
    12: 'STORE_1_587', 13: 'STORE_1_714', 14: 'STORE_1_727'
}

# Создаем DataFrame с результатами
lgbm_results_df = pd.DataFrame({
    'date': test_features_df['date'].values,
    'item_id_encoded': test_features_df['item_id_encoded'].values, # Закодированный ID
    'cnt': y_test.values,
    'yhat_lgbm': y_pred_lgb # Или другой прогноз, если нужно
})
# Добавляем исходный item_id для группировки при оценке
lgbm_results_df['item_id'] = lgbm_results_df['item_id_encoded'].map(item_id_map_reverse)


# --- Сводная таблица метрик LightGBM ---
lgbm_simplified_summary_df = pd.DataFrame() # Инициализируем на случай ошибок
if 'item_id' in lgbm_results_df.columns and not lgbm_results_df['item_id'].isnull().all():
    lgbm_results_summary = {}
    min_test_date = lgbm_results_df['date'].min()
    horizons = {'Week': 7, 'Month': 30, 'Quarter': 90}
    print("\nРасчет метрик для LightGBM (упрощенные признаки)...")
    for name, days in horizons.items():
        test_horizon = lgbm_results_df[lgbm_results_df['date'] <= min_test_date + pd.Timedelta(days=days-1)]
        if test_horizon.empty or test_horizon['item_id'].isnull().all():
            print(f"Предупреждение: Нет данных для горизонта '{name}'.")
            lgbm_results_summary[name] = pd.Series({'MAE': np.nan, 'RMSE': np.nan, 'sMAPE': np.nan, 'R2': np.nan})
            continue
        try:
            horizon_metrics_df = test_horizon.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_lgbm'])))
            lgbm_results_summary[name] = horizon_metrics_df.mean()
        except Exception as e:
            print(f"Ошибка при расчете метрик для горизонта '{name}': {e}")
            lgbm_results_summary[name] = pd.Series({'MAE': np.nan, 'RMSE': np.nan, 'sMAPE': np.nan, 'R2': np.nan})
    lgbm_simplified_summary_df = pd.DataFrame(lgbm_results_summary) # Переприсваиваем
else:
     print("Ошибка: Не удалось получить/декодировать item_id для расчета метрик.")


print("\n--- Сводная таблица метрик LightGBM (упрощенные признаки) ---")
print(lgbm_simplified_summary_df)


# Дополнительно: Важность признаков
try:
    print("\n--- Важность признаков LightGBM (упрощенные признаки) ---")
    feature_importance_df = pd.DataFrame({
        'feature': model_lgb_simplified.feature_name_,
        'importance': model_lgb_simplified.feature_importances_
    }).sort_values(by='importance', ascending=False)
    print(feature_importance_df.head(20))
except Exception as e:
    print(f"Не удалось получить важность признаков: {e}")


--- Обучение LightGBM (упрощенные признаки) ---
Обучение финальной модели LGBM...
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
Обучение LightGBM завершено за 0.33 сек.

Расчет метрик для LightGBM (упрощенные признаки)...

--- Сводная таблица метрик LightGBM (упрощенные признаки) ---
            Week      Month    Quarter
MAE     4.365823   4.390217   5.284014
RMSE    5.363034   5.526117   7.226407
sMAPE  85.884854  82.574130  86.079746
R2     -0.122750   0.016399   0.019858

--- Важность признаков LightGBM (упрощенные признаки) ---
             feature  importance
7          dayofyear         401
6         weekofyear         265
5         dayofmonth         131
20   item_id_encoded         124
19   cnt_roll_std_28         104
14   cnt_roll_mean_7         101
3         sell_price          96
18  cnt_roll_mean_28          81
8          cnt_lag_7          71
0               wday          53
16  c

### Подбор параметров LBGM

In [ ]:
# Ручной подбор параметров LGBM - Упрощенные признаки
print("--- Ручной подбор параметров (Grid Search) для LightGBM (упрощенные признаки) ---")

# --- Определяем сетку параметров ---
param_grid = {
    'num_leaves': [15, 31, 63],
    'learning_rate': [0.02, 0.05, 0.1],
    'feature_fraction': [0.7, 0.8, 0.9],
    'bagging_fraction': [0.7, 0.8, 0.9]
}

# Фиксированные параметры
base_params = {
    'objective': 'regression_l1', 'metric': 'mae', 'n_estimators': 2000,
    'bagging_freq': 1, 'verbose': -1, 'n_jobs': -1, 'seed': 42,
    'boosting_type': 'gbdt',
}
early_stopping_rounds = 50

# --- Перебор параметров ---
results = []
best_mae = np.inf
best_params = {}
best_iteration = 0

keys, values = zip(*param_grid.items())
param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
print(f"Всего комбинаций для перебора: {len(param_combinations)}")

start_grid_search_time = time.time()

for params_combo in tqdm(param_combinations, desc="Grid Search Progress"):
    current_params = base_params.copy()
    current_params.update(params_combo)
    try:
        model_lgb_gs = lgb.LGBMRegressor(**current_params)
        model_lgb_gs.fit(X_train, y_train,
                         eval_set=[(X_test, y_test)],
                         eval_metric='mae',
                         callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_rounds, verbose=False)],
                         categorical_feature=CATEGORICAL_FEATURES)

        current_best_iter = model_lgb_gs.best_iteration_ if model_lgb_gs.best_iteration_ is not None else current_params['n_estimators']
        if current_best_iter is None or current_best_iter <= 0: current_best_iter = 1 # Ensure positive iteration
        y_pred_gs = model_lgb_gs.predict(X_test, num_iteration=current_best_iter)
        y_pred_gs = np.maximum(0, y_pred_gs)
        mae_score = mean_absolute_error(y_test, y_pred_gs)
        results.append({'params': params_combo, 'mae': mae_score, 'best_iteration': current_best_iter})

        if mae_score < best_mae:
            best_mae = mae_score; best_params = params_combo; best_iteration = current_best_iter

    except Exception as e:
        print(f"  Error processing combo {params_combo}: {e}")
        results.append({'params': params_combo, 'mae': np.inf, 'best_iteration': None})


end_grid_search_time = time.time()

--- Ручной подбор параметров (Grid Search) для LightGBM (упрощенные признаки) ---
Всего комбинаций для перебора: 81


Grid Search Progress:   0%|          | 0/81 [00:00<?, ?it/s]

[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categoric

In [65]:
# выведу отдельно, чтобы скрыть warning'и
print(f"\n--- Grid Search завершен за {end_grid_search_time - start_grid_search_time:.2f} сек. ---")
print(f"Лучшая MAE на тестовом наборе (90 дней): {best_mae:.4f}")
print("Лучшие параметры:")
print(best_params)
print(f"Лучшее количество итераций: {best_iteration}")


--- Grid Search завершен за 121.12 сек. ---
Лучшая MAE на тестовом наборе (90 дней): 5.0243
Лучшие параметры:
{'num_leaves': 15, 'learning_rate': 0.1, 'feature_fraction': 0.8, 'bagging_fraction': 0.9}
Лучшее количество итераций: 127


In [77]:
# --- Обучение финальной модели с лучшими параметрами ---
print("\n--- Обучение финальной модели LGBM с лучшими параметрами (упрощенные признаки) ---")
final_params = base_params.copy()
final_params.update(best_params)
final_params['n_estimators'] = best_iteration if best_iteration > 0 else 10 # Используем найденное число деревьев
if 'metric' in final_params: del final_params['metric'] # Удаляем метрику для финального обучения

start_final_fit_time = time.time()
model_lgb_final_simplified = lgb.LGBMRegressor(**final_params)
# Обучаем на ВСЕХ X_train, y_train с КАТЕГОРИЯМИ
model_lgb_final_simplified.fit(X_train, y_train, categorical_feature=CATEGORICAL_FEATURES)
end_final_fit_time = time.time()
print(f"Обучение финальной модели завершено за {end_final_fit_time - start_final_fit_time:.2f} сек.")


# --- Прогнозирование и Оценка финальной УПРОЩЕННОЙ модели ---
print("\n--- Генерация прогнозов финальной модели LightGBM (упрощенные признаки) ---")
start_predict_time = time.time()
# Прогнозируем на X_test
y_pred_lgb_final = model_lgb_final_simplified.predict(X_test)
y_pred_lgb_final = np.maximum(0, y_pred_lgb_final)
end_predict_time = time.time()
print(f"Прогнозирование завершено за {end_predict_time - start_predict_time:.2f} сек.")

# --- Оценка ---
# Создаем обратный маппинг для item_id (из хардкода в Ячейке 6.1)
item_id_map_reverse = {
    0: 'STORE_1_064', 1: 'STORE_1_065', 2: 'STORE_1_090', 3: 'STORE_1_252',
    4: 'STORE_1_325', 5: 'STORE_1_339', 6: 'STORE_1_376', 7: 'STORE_1_546',
    8: 'STORE_1_547', 9: 'STORE_1_555', 10: 'STORE_1_584', 11: 'STORE_1_586',
    12: 'STORE_1_587', 13: 'STORE_1_714', 14: 'STORE_1_727'
}

# Создаем DataFrame с результатами, используя test_features_df для получения item_id_encoded и date
lgbm_final_simplified_results_df = pd.DataFrame({
    'date': test_features_df['date'].values,
    'item_id_encoded': test_features_df['item_id_encoded'].values, # Закодированный ID
    'cnt': y_test.values, # Фактические значения из y_test
    'yhat_lgbm_final_simpl': y_pred_lgb_final # Прогнозы
})
# Добавляем исходный строковый item_id для группировки при оценке
lgbm_final_simplified_results_df['item_id'] = lgbm_final_simplified_results_df['item_id_encoded'].map(item_id_map_reverse)

# --- Сводная таблица метрик ---
lgbm_final_simplified_summary_df = pd.DataFrame() # Инициализация
if 'item_id' in lgbm_final_simplified_results_df.columns and not lgbm_final_simplified_results_df['item_id'].isnull().all():
    lgbm_results_summary = {}
    min_test_date = lgbm_final_simplified_results_df['date'].min()
    # Определяем горизонты (можно взять из констант FH_*)
    horizons = {'Week': 7, 'Month': 30, 'Quarter': 90}
    print("\nРасчет метрик для финальной упрощенной модели LightGBM...")
    for name, days in horizons.items():
        test_horizon = lgbm_final_simplified_results_df[lgbm_final_simplified_results_df['date'] <= min_test_date + pd.Timedelta(days=days-1)]
        if test_horizon.empty or test_horizon['item_id'].isnull().all():
            print(f"Предупреждение: Нет данных для горизонта '{name}'.")
            lgbm_results_summary[name] = pd.Series({'MAE': np.nan, 'RMSE': np.nan, 'sMAPE': np.nan, 'R2': np.nan})
            continue
        try:
            horizon_metrics_df = test_horizon.groupby('item_id').apply(lambda x: pd.Series(calculate_metrics(x['cnt'], x['yhat_lgbm_final_simpl']))) # Исправлено имя прогноза
            lgbm_results_summary[name] = horizon_metrics_df.mean()
        except Exception as e:
            print(f"Ошибка при расчете метрик для горизонта '{name}': {e}")
            lgbm_results_summary[name] = pd.Series({'MAE': np.nan, 'RMSE': np.nan, 'sMAPE': np.nan, 'R2': np.nan})
    lgbm_final_simplified_summary_df = pd.DataFrame(lgbm_results_summary)
else:
     print("Ошибка: Не удалось получить/декодировать item_id для расчета метрик.")

print("\n--- Сводная таблица метрик финальной УПРОЩЕННОЙ модели LightGBM ---")
print(lgbm_final_simplified_summary_df)

# --- Важность признаков ---
try:
    print("\n--- Важность признаков финальной УПРОЩЕННОЙ модели LightGBM ---")
    feature_importance_df_final_simpl = pd.DataFrame({
        'feature': model_lgb_final_simplified.feature_name_,
        'importance': model_lgb_final_simplified.feature_importances_
    }).sort_values(by='importance', ascending=False)
    print(feature_importance_df_final_simpl.head(20))
except Exception as e:
    print(f"Не удалось получить важность признаков: {e}")


--- Обучение финальной модели LGBM с лучшими параметрами (упрощенные признаки) ---
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
Обучение финальной модели завершено за 0.38 сек.

--- Генерация прогнозов финальной модели LightGBM (упрощенные признаки) ---
Прогнозирование завершено за 0.00 сек.

Расчет метрик для финальной упрощенной модели LightGBM...

--- Сводная таблица метрик финальной УПРОЩЕННОЙ модели LightGBM ---
            Week      Month    Quarter
MAE     3.955409   4.310566   5.024306
RMSE    5.019303   5.454778   7.044391
sMAPE  69.375718  77.257021  79.240610
R2      0.021343   0.023109   0.030771

--- Важность признаков финальной УПРОЩЕННОЙ модели LightGBM ---
             feature  importance
7          dayofyear         367
6         weekofyear         250
3         sell_price         179
19   cnt_roll_std_28         172
18  cnt_roll_mean_28         142
14   cnt_roll_mean_7       

## Итоговые метрики по всем протестированнным моделям

In [67]:
# --- Создание ФИНАЛЬНЫХ сводных таблиц сравнения ВСЕХ моделей по горизонтам (отсортировано по MAE) ---

# Добавляем результаты финальной модели LGBM к списку
# Убедимся, что lgbm_final_summary_df существует
if 'lgbm_final_simplified_summary_df' not in locals():
     print("Ошибка: Не найден DataFrame lgbm_final_simplified_summary_df. Выполните шаг 6.3.")
else:
    model_summaries_final = {
        'Baseline': 'baseline_summary_df',
        'ETS': 'ets_summary_df',
        'Prophet': 'prophet_summary_df',
        'AutoARIMA': 'autoarima_summary_df',
        'TBATS': 'tbats_summary_df',
        'Orbit': 'orbit_summary_df',
        'LGBM_simplified_tuned': 'lgbm_final_simplified_summary_df' # Добавляем новую модель
    }

    # Проверяем наличие всех DataFrame'ов
    missing_dfs_final = []
    for df_name in model_summaries_final.values():
        if df_name not in locals():
            missing_dfs_final.append(df_name)

    if missing_dfs_final:
        print(f"Ошибка: Не найдены DataFrame'ы: {', '.join(missing_dfs_final)}")
    else:
        # --- Таблица для Недельного горизонта (FH=7) ---
        week_metrics_list_final = {}
        for model_name, df_name in model_summaries_final.items():
            df = locals()[df_name]
            if 'Week' in df.columns and not df['Week'].isnull().all(): # Добавлена проверка на NaN
                week_metrics_list_final[model_name] = df['Week']
            else:
                print(f"Предупреждение: Данные для 'Week' отсутствуют или содержат NaN в {df_name}")

        # Проверяем, есть ли что конкатенировать
        if week_metrics_list_final:
             metrics_week_comparison_final = pd.concat(week_metrics_list_final, axis=1)
             # Сортируем по MAE
             metrics_week_final_sorted = metrics_week_comparison_final.sort_values(by='MAE', axis=1)
             print("\n--- Итоговое сравнение моделей: Горизонт - Неделя (Отсортировано по MAE) ---")
             print(metrics_week_final_sorted.round(3))
        else:
            print("\nНедостаточно данных для сравнения моделей на недельном горизонте.")


        # --- Таблица для Месячного горизонта (FH=30) ---
        month_metrics_list_final = {}
        for model_name, df_name in model_summaries_final.items():
            df = locals()[df_name]
            if 'Month' in df.columns and not df['Month'].isnull().all():
                month_metrics_list_final[model_name] = df['Month']
            else:
                print(f"Предупреждение: Данные для 'Month' отсутствуют или содержат NaN в {df_name}")

        if month_metrics_list_final:
            metrics_month_comparison_final = pd.concat(month_metrics_list_final, axis=1)
            metrics_month_final_sorted = metrics_month_comparison_final.sort_values(by='MAE', axis=1)
            print("\n--- Итоговое сравнение моделей: Горизонт - Месяц (Отсортировано по MAE) ---")
            print(metrics_month_final_sorted.round(3))
        else:
            print("\nНедостаточно данных для сравнения моделей на месячном горизонте.")


        # --- Таблица для Квартального горизонта (FH=90) ---
        quarter_metrics_list_final = {}
        for model_name, df_name in model_summaries_final.items():
            df = locals()[df_name]
            if 'Quarter' in df.columns and not df['Quarter'].isnull().all():
                quarter_metrics_list_final[model_name] = df['Quarter']
            else:
                print(f"Предупреждение: Данные для 'Quarter' отсутствуют или содержат NaN в {df_name}")

        if quarter_metrics_list_final:
            metrics_quarter_comparison_final = pd.concat(quarter_metrics_list_final, axis=1)
            metrics_quarter_final_sorted = metrics_quarter_comparison_final.sort_values(by='MAE', axis=1)
            print("\n--- Итоговое сравнение моделей: Горизонт - Квартал (Отсортировано по MAE) ---")
            print(metrics_quarter_final_sorted.round(3))
        else:
             print("\nНедостаточно данных для сравнения моделей на квартальном горизонте.")


--- Итоговое сравнение моделей: Горизонт - Неделя (Отсортировано по MAE) ---
       LGBM_simplified_tuned   Orbit     ETS   TBATS  AutoARIMA  Baseline  Prophet
MAE                    3.955   4.064   4.149   4.330      4.621     5.181    5.885
RMSE                   5.019   4.828   4.872   4.994      5.298     6.515    6.649
sMAPE                 69.376  79.971  76.182  77.680     79.628    69.216   86.743
R2                     0.021   0.113   0.098   0.063     -0.020    -0.820   -1.070

--- Итоговое сравнение моделей: Горизонт - Месяц (Отсортировано по MAE) ---
       LGBM_simplified_tuned   TBATS   Orbit     ETS  AutoARIMA  Prophet  Baseline
MAE                    4.311   4.608   4.626   4.785      4.911    5.265     5.696
RMSE                   5.455   5.735   5.754   5.833      5.839    6.392     7.372
sMAPE                 77.257  76.456  84.786  79.566     81.026   84.176    70.952
R2                     0.023   0.012  -0.024   0.076      0.035   -0.302    -0.687

--- Итоговое с

## Сохранение результатов и моделей

In [ ]:
# Сохранение сводных таблиц метрик
# Директория для сохранения результатов
results_dir = 'results'
os.makedirs(results_dir, exist_ok=True) # Создаем директорию, если её нет

# Словарь с таблицами для сохранения
metrics_tables_to_save = {
    'week': 'metrics_week_final_sorted',
    'month': 'metrics_month_final_sorted',
    'quarter': 'metrics_quarter_final_sorted'
}

print(f"\n--- Сохранение сводных таблиц метрик в '{results_dir}/' ---")

for horizon, df_name in metrics_tables_to_save.items():
    try:
        # Получаем DataFrame по его имени из локальных переменных
        df_to_save = locals()[df_name]
        if isinstance(df_to_save, pd.DataFrame):
            filepath = os.path.join(results_dir, f'comparison_metrics_{horizon}_sorted.csv')
            df_to_save.to_csv(filepath)
            print(f"Таблица для горизонта '{horizon}' сохранена в: {filepath}")
        else:
            print(f"Переменная {df_name} не является DataFrame, пропуск.")
    except KeyError:
        print(f"Ошибка: DataFrame '{df_name}' не найден. Не могу сохранить метрики для '{horizon}'.")
    except Exception as e:
        print(f"Ошибка при сохранении метрик для '{horizon}': {e}")


--- Сохранение сводных таблиц метрик в 'results/' ---
Таблица для горизонта 'week' сохранена в: results/comparison_metrics_week_sorted.csv
Таблица для горизонта 'month' сохранена в: results/comparison_metrics_month_sorted.csv
Таблица для горизонта 'quarter' сохранена в: results/comparison_metrics_quarter_sorted.csv


In [ ]:
# Сохранение обученных моделей

# Директория для сохранения моделей
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True) # Создаем директорию

# Список моделей/словарей для сохранения
models_to_save = {
    'ets': 'ets_models',
    'prophet': 'prophet_models',
    # 'autoarima': 'autoarima_models', АвтоАриму не сохраняем, т.к. файл получается ~1 ГБ
    'tbats': 'tbats_models',
    'orbit': 'orbit_models',
    'lgb_final_simplified': 'model_lgb_final_simplified'
}

print(f"\n--- Сохранение обученных моделей в '{models_dir}/' ---")

for model_key, model_var_name in models_to_save.items():
    try:
        model_object = locals()[model_var_name]
        if model_object is not None:
            filepath = os.path.join(models_dir, f'{model_key}.joblib')
            joblib.dump(model_object, filepath)
            print(f"Модель(и) '{model_key}' сохранена в: {filepath}")
        else:
             print(f"Объект '{model_var_name}' равен None, пропуск сохранения.")

    except KeyError:
        print(f"Ошибка: Объект '{model_var_name}' не найден. Не могу сохранить модель '{model_key}'.")
    except Exception as e:
        # Обработка возможных ошибок сериализации (особенно для Prophet/Orbit)
        print(f"(!) Ошибка при сохранении модели '{model_key}': {e}")
        print(f"    Модели Prophet/Orbit могут иметь проблемы с сериализацией.")

# Особое примечание для LightGBM:
# Модели LightGBM также можно сохранять собственным методом save_model,
# который создает текстовый файл, более устойчивый к изменениям версий.
try:
    if 'model_lgb_final_simplified' in locals() and model_lgb_final_simplified is not None:
        lgbm_native_path = os.path.join(models_dir, 'model_lgb_final_simplified.txt')
        model_lgb_final_simplified.booster_.save_model(lgbm_native_path)
        print(f"Модель LGBM также сохранена в нативном формате: {lgbm_native_path}")
except Exception as e:
    print(f"Ошибка при сохранении LGBM в нативном формате: {e}")


--- Сохранение обученных моделей в 'models/' ---
Модель(и) 'ets' сохранена в: models/ets.joblib
Модель(и) 'prophet' сохранена в: models/prophet.joblib
Модель(и) 'tbats' сохранена в: models/tbats.joblib
Модель(и) 'orbit' сохранена в: models/orbit.joblib
Модель(и) 'lgb_final_simplified' сохранена в: models/lgb_final_simplified.joblib
Модель LGBM также сохранена в нативном формате: models/model_lgb_final_simplified.txt


## Выводы

Отличная работа! Мы прошли долгий путь от анализа данных до обучения, сравнения и оптимизации нескольких моделей. Давайте подведем итоги и сделаем финальные выводы.

**Резюме процесса:**

1.  **EDA:** Глубоко изучили данные `STORE_1`, выявили тренды, сильную недельную сезонность, влияние праздников (которое, как оказалось позже, хорошо кодируется календарными признаками), нестационарность рядов и остаточную автокорреляцию после простой декомпозиции. Обработали пропуски.
2.  **Подготовка:** Разделили данные на обучающий и 90-дневный тестовый наборы. Определили метрики (MAE, RMSE, sMAPE, R2).
3.  **Моделирование (Этап 1 - Широкий поиск):**
    *   Создали `Baseline` (сезонный наивный).
    *   Обучили статистические модели: `ETS`, `AutoARIMA`, `TBATS`.
    *   Обучили модели с регрессорами: `Prophet`, `Orbit (DLT)`.
    *   Обучили ML модель: `LightGBM` с обширным набором признаков (календарные, лаги, окна, события, цена, ID товара и т.д.).
4.  **Сравнение (Этап 1):** Сравнили все модели на горизонтах неделя/месяц/квартал. LightGBM показал очень хорошие результаты по MAE/RMSE, но высокий sMAPE. ETS, TBATS, Orbit были конкурентоспособны на коротких горизонтах. Prophet выделился на квартальном горизонте (но все равно уступал LGBM).
5.  **Оптимизация LGBM (Этап 2):**
    *   Проанализировали важность признаков для LGBM. Обнаружили, что явные признаки событий почти не используются, а `dayofyear`, `weekofyear`, `dayofmonth` доминируют. `item_id`, лаги, окна и цена также важны. `month` и `cashback` оказались неважными.
    *   **Упростили Feature Engineering:** Удалили признаки событий, `month`, `cashback`. Заменили `LabelEncoder` для `item_id` на извлечение последних 3 цифр (создав `item_id_numeric`) и решили **считать его категориальным признаком** для LGBM (т.к. коды 0-14 теперь плотные).
    *   **Подобрали параметры:** Выполнили Grid Search для `num_leaves`, `learning_rate`, `feature_fraction`, `bagging_fraction`, найдя оптимальные значения и количество итераций (`n_estimators=57`).
    *   Обучили финальную **`LGBM_simplified_tuned`**.
6.  **Финальное сравнение:** Сравнили все исходные модели с финальной оптимизированной моделью LGBM.
7.  **Разработка класса:** Реализовали класс `DemandForecasterLGBM`, инкапсулирующий упрощенный процесс создания признаков, обучения, сохранения/загрузки необходимых компонентов (модель, фичи, категории, история, события) и прогнозирования на новых данных. Прошли через несколько итераций отладки класса.

**Анализ итоговых результатов:**

1.  **Лидерство `LGBM_simplified_tuned`:** Финальная, оптимизированная и упрощенная модель LightGBM стала **абсолютным лидером по MAE и RMSE на всех трех горизонтах**.
    *   **Неделя:** MAE 3.955 (лучший).
    *   **Месяц:** MAE 4.311 (лучший, со значительным отрывом).
    *   **Квартал:** MAE 5.024 (лучший, с огромным отрывом от следующего - Prophet с 6.831).
2.  **Улучшение sMAPE для LGBM:** Упрощение признаков и, возможно, использование плотных кодов для `item_id` **резко улучшили sMAPE** для LightGBM. Теперь он показывает лучший или второй лучший sMAPE на всех горизонтах (69.4 на неделе, 77.3 на месяце, 79.2 на квартале), уступая только `Baseline`. Это решает основную проблему предыдущей версии LGBM.
3.  **R2:** Упрощенный LGBM показывает **положительный R2** на всех горизонтах (хотя и небольшой: 0.021, 0.023, 0.031). Это единственная модель, которая стабильно лучше константного прогноза среднего на всех периодах, включая квартальный.
4.  **Другие модели:**
    *   `Orbit` и `ETS` остаются сильными конкурентами на недельном горизонте по MAE/RMSE, но уступают на более длинных.
    *   `TBATS` хорошо показал себя на месячном горизонте (второе место по MAE после LGBM).
    *   `Prophet` интересен тем, что стал вторым по MAE на квартальном горизонте, но все равно значительно хуже LGBM.
    *   `AutoARIMA` показала средние результаты.
    *   `Baseline` подтвердил свою роль - худший по абсолютным метрикам, но часто лучший по sMAPE.

**Финальные выводы:**

1.  **Лучшая модель:** Для данной задачи и данных **оптимизированная модель LightGBM с упрощенным набором признаков (`LGBM_simplified_tuned`) является наилучшим выбором**. Она обеспечивает самую высокую точность по абсолютным ошибкам (MAE/RMSE) на всех горизонтах и имеет конкурентоспособную относительную ошибку (sMAPE), а также высокую скорость обучения и прогнозирования.
2.  **Ценность Feature Engineering и Оптимизации:** Процесс показал, что:
    *   Создание релевантных признаков (календарные, лаги, окна, цена, ID товара) критически важно для успеха ML-моделей.
    *   Анализ важности признаков позволяет **упростить модель**, убрав избыточные или неважные фичи (как признаки событий), что может **улучшить** не только интерпретируемость, но и некоторые метрики (как sMAPE в нашем случае) и скорость.
    *   Подбор гиперпараметров (даже простой Grid Search) может дать дополнительный прирост качества.
3.  **Кодирование ID:** Использование простого числового ID (0-14) и указание его как категориального признака для LGBM сработало хорошо и убрало соответствующие предупреждения.
4.  **Горизонт прогнозирования:** Точность прогноза ожидаемо падает с увеличением горизонта. Долгосрочное (квартал) прогнозирование остается сложной задачей с низкой общей предсказуемостью (низкий R2).
5.  **Класс для инференса:** Создание класса потребовало тщательной проработки логики сохранения/загрузки состояния модели и обработки входных данных на этапе `predict` (особенно расчета лаговых признаков с использованием сохраненной истории).

Таким образом, итоговым решением является использование **финальной модели LightGBM**, обученной на упрощенном наборе признаков с оптимальными параметрами, инкапсулированной в класс `DemandForecasterLGBM` для удобства использования и инференса.